# Importing Contigencies

In [ ]:
!pip install --upgrade torch torchvision timm

In [ ]:
!pip install timm

In [ ]:
from google.colab import drive
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from PIL import Image
import random
import pandas as pd

In [ ]:
drive.mount('/content/drive')

# Data preprocessing
## 1) Unzipping the MRI Images

In [ ]:
# Path to your zip file
zip_path = '/content/drive/My Drive/Colab Notebooks/mri_images.zip'
# Create a directory to extract the files
extract_path = '/content/mri_images'
os.makedirs(extract_path, exist_ok=True)

In [ ]:
# Get the name of the zip file without the path
zip_filename = os.path.basename(zip_path)

# Get the name of the extracted folder (remove .zip extension)
extracted_folder = os.path.splitext(zip_filename)[0]


In [ ]:
# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

print(f"Zip file: {zip_filename}")
print(f"Extracted folder: {extracted_folder}")
print(f"Files extracted to: {extract_path}")


In [ ]:
# Define the path to the glioma folder
glioma_path = os.path.join('Testing', 'glioma')

# Verify the path exists
if os.path.exists(glioma_path):
    print(f"Successfully located the glioma folder at: {glioma_path}")
else:
    print("Couldn't find the glioma folder. Please check the path.")

#### Viewing a Random Image:

In [ ]:
# Get a list of all image files in the glioma folder
image_files = [f for f in os.listdir(glioma_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

if image_files:
    # Select a random image
    random_image = random.choice(image_files)
    image_path = os.path.join(glioma_path, random_image)

    # Open and display the image
    img = Image.open(image_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Sample Glioma Image: {random_image}")
    plt.show()
else:
    print("No image files found in the glioma folder.")

In [ ]:
# Get a list of all image files in the glioma folder
image_files = [f for f in os.listdir(glioma_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

if image_files:
    # Select a random image
    random_image = random.choice(image_files)
    image_path = os.path.join(glioma_path, random_image)

    # Open the image
    img = Image.open(image_path)

    # Convert the image to a numpy array
    img_array = np.array(img)

    # Display the image
    plt.figure(figsize=(8, 8))
    plt.imshow(img_array, cmap='gray')  # Use 'gray' colormap
    plt.axis('off')
    plt.title(f"Sample Glioma Image: {random_image}")
    plt.show()
else:
    print("No image files found in the glioma folder.")

## 2) Resizing all the images to ResNet-50 input size and Saving them in a new folder

In [ ]:
def resize_images(input_folder, output_folder, target_size=(224, 224)):
    """
    Resize all images in the input folder and save them to the output folder.

    Args:
    input_folder (str): Path to the folder containing original images
    output_folder (str): Path to the folder where resized images will be saved
    target_size (tuple): The target size for the images (width, height)
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            try:
                img = Image.open(os.path.join(input_folder, filename))
                img = img.resize(target_size, Image.LANCZOS)
                img.save(os.path.join(output_folder, filename))
                print(f"Resized {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

# Define the size you want for all images
target_size = (224, 224)  # This is a common size for many CNN architectures

In [ ]:
# Define the base directories
base_input_dir = 'Training'  # Adjust this if your folder structure is different
base_output_dir = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'

# List of tumor types
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Resize images for each tumor type
for tumor_type in tumor_types:
    input_folder = os.path.join(base_input_dir, tumor_type)
    output_folder = os.path.join(base_output_dir, tumor_type)

    print(f"Resizing images in {tumor_type} folder...")
    resize_images(input_folder, output_folder, target_size)

print("Image resizing complete!")

In [ ]:
# Define the base directories
base_input_dir = 'Testing'  # Adjust this if your folder structure is different
base_output_dir = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'

# List of tumor types
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Resize images for each tumor type
for tumor_type in tumor_types:
    input_folder = os.path.join(base_input_dir, tumor_type)
    output_folder = os.path.join(base_output_dir, tumor_type)

    print(f"Resizing images in {tumor_type} folder...")
    resize_images(input_folder, output_folder, target_size)

print("Image resizing complete!")

### Loading the resized folders from google drive:

In [ ]:
# Mount Google Drive if not already mounted
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define the base directory in Google Drive
base_output_dir = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'

# List of tumor types
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Choose a random tumor type and image
tumor_type = random.choice(tumor_types)
resized_folder = os.path.join(base_output_dir, tumor_type)

# Check if the folder exists
if not os.path.exists(resized_folder):
    print(f"Error: The folder {resized_folder} does not exist in your Google Drive.")
else:
    image_files = [f for f in os.listdir(resized_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    if not image_files:
        print(f"Error: No image files found in {resized_folder}")
    else:
        random_image = random.choice(image_files)

        # Display the image
        img_path = os.path.join(resized_folder, random_image)
        img = Image.open(img_path)
        img_array = np.array(img)

        plt.figure(figsize=(8, 8))
        plt.imshow(img_array, cmap='gray')
        plt.axis('off')
        plt.title(f"Resized {tumor_type} Image: {random_image}\nSize: {img_array.shape}")
        plt.show()

        print(f"Image size: {img_array.shape}")

In [ ]:
# Mount Google Drive if not already mounted
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define the base directory in Google Drive
base_output_dir = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'

# List of tumor types
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Choose a random tumor type and image
tumor_type = random.choice(tumor_types)
resized_folder = os.path.join(base_output_dir, tumor_type)

# Check if the folder exists
if not os.path.exists(resized_folder):
    print(f"Error: The folder {resized_folder} does not exist in your Google Drive.")
else:
    image_files = [f for f in os.listdir(resized_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    if not image_files:
        print(f"Error: No image files found in {resized_folder}")
    else:
        random_image = random.choice(image_files)

        # Display the image
        img_path = os.path.join(resized_folder, random_image)
        img = Image.open(img_path)
        img_array = np.array(img)

        plt.figure(figsize=(8, 8))
        plt.imshow(img_array, cmap='gray')
        plt.axis('off')
        plt.title(f"Resized {tumor_type} Image: {random_image}\nSize: {img_array.shape}")
        plt.show()

        print(f"Image size: {img_array.shape}")

# PyTorch Data Pre-Processing

## 1) DataSet & DataLoader

Importing PyTorch contigencies:

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from scipy.ndimage import gaussian_filter, map_coordinates
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as TF

Defining the data sets:

In [ ]:
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

DataSet Class:

## 2) Data Augmentation

1. Random Crop
2. Random Flip
3. Random Rotate
4. Random Brightness
5. Random Contrast
6. On the fly augmentation during training (saves memory)

__Important__

1) Avoid extreme rotations or flips that could change the anatomical orientation.

2) Be cautious with color-based augmentations, as MRI intensity values often have specific meanings.

3)Maintain the overall structure and proportions of the brain.

1. Slight rotations (within ±10 degrees)
2. Small shifts (translations)
3. Zoom in/out (within a small range)
4. Minimal brightness and contrast adjustments
5. Gaussian noise addition (to simulate image noise)
6. Elastic deformations (subtle warping)  

In [ ]:
class RandomMRIAugmentation(nn.Module):
    def __init__(self, rotation_range=10, translation_range=0.1, zoom_range=0.1, noise_factor=0.05, p=0.5):
        super().__init__()
        self.rotation_range = rotation_range
        self.translation_range = translation_range
        self.zoom_range = zoom_range
        self.noise_factor = noise_factor
        self.p = p  # Probability of applying each augmentation

    def forward(self, img):
        # Ensure input is a tensor
        if not isinstance(img, torch.Tensor):
            img = TF.to_tensor(img)

        # Random rotation
        if random.random() < self.p:
            angle = random.uniform(-self.rotation_range, self.rotation_range)
            img = TF.rotate(img, angle)

        # Random translation
        if random.random() < self.p:
            translate = [random.uniform(-self.translation_range, self.translation_range) for _ in range(2)]
            img = TF.affine(img, angle=0, translate=translate, scale=1, shear=0)

        # Random zoom
        if random.random() < self.p:
            scale = random.uniform(1-self.zoom_range, 1+self.zoom_range)
            img = TF.affine(img, angle=0, translate=(0,0), scale=scale, shear=0)

        # Add Gaussian noise
        if random.random() < self.p:
            noise = torch.randn_like(img) * self.noise_factor
            img = img + noise
            img = torch.clamp(img, 0, 1)

        return img

def elastic_transform(image, alpha=1000, sigma=30, alpha_affine=30):
    """Elastic deformation of images as described in [Simard2003]."""
    random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]

    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha

    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

class RandomElasticDeformation(nn.Module):
    def __init__(self, p=0.2, alpha=1000, sigma=30, alpha_affine=30):
        super().__init__()
        self.p = p
        self.alpha = alpha
        self.sigma = sigma
        self.alpha_affine = alpha_affine

    def forward(self, img):
        if random.random() < self.p:
            if isinstance(img, torch.Tensor):
                img = img.numpy()
            img = elastic_transform(img, self.alpha, self.sigma, self.alpha_affine)
            img = torch.from_numpy(img)
        return img

def get_mri_augmentation(p_transform=0.5, p_elastic=0.2):
    return transforms.Compose([
        RandomMRIAugmentation(rotation_range=10, translation_range=0.1, zoom_range=0.1, noise_factor=0.05, p=p_transform),
        RandomElasticDeformation(p=p_elastic),
        transforms.Normalize(mean=[0.485], std=[0.229])  # Adjust these values based on your MRI data statistics
    ])

In [ ]:
class MRIDataset(Dataset):
    def __init__(self, folder_path, tumor_types, transform=None, augment=False):
        self.folder_path = folder_path
        self.tumor_types = tumor_types
        self.transform = transform
        self.augment = augment
        self.image_paths = []
        self.labels = []

        for label, tumor_type in enumerate(tumor_types):
            tumor_folder = os.path.join(folder_path, tumor_type)
            for img_name in os.listdir(tumor_folder):
                self.image_paths.append(os.path.join(tumor_folder, img_name))
                self.labels.append(label)

        if self.augment:
            self.aug_transform = get_mri_augmentation(p_transform=0.5, p_elastic=0.2)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.augment:
            image = self.aug_transform(image)
        elif self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
def create_mri_datasets(train_folder, test_folder, tumor_types, val_split=0.2, batch_size=32):
    # Define base transform for validation and test sets
    base_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])  # Adjust based on your MRI data
    ])

    # Create datasets
    full_train_dataset = MRIDataset(train_folder, tumor_types, augment=True)
    test_dataset = MRIDataset(test_folder, tumor_types, transform=base_transform)

    # Split the training dataset into train and validation
    train_indices, val_indices = train_test_split(
        range(len(full_train_dataset)),
        test_size=val_split,
        stratify=full_train_dataset.labels,
        random_state=42
    )

    train_dataset = torch.utils.data.Subset(full_train_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(full_train_dataset, val_indices)

    # Override the transform for the validation set
    val_dataset.dataset.augment = False
    val_dataset.dataset.transform = base_transform

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader


In [ ]:
# Usage
train_loader, val_loader, test_loader = create_mri_datasets(train_folder, test_folder, tumor_types)


# Model 1: ResNet50

In [ ]:
!pip install grad-cam

In [ ]:
import time
from torch.utils.data import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

Defining the model:


In [ ]:
def initialize_model(num_classes):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def train_model(model, train_loader, val_loader, num_epochs=100, patience=10):
    start_time = time.time()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

    best_val_loss = float('inf')
    best_val_acc = 0.0
    epochs_no_improve = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
             best_val_loss = val_loss
             best_val_acc = val_acc
             epochs_no_improve = 0
             best_model = model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            model.load_state_dict(best_model)
            break

    end_time = time.time()
    training_time = end_time - start_time
    return model, best_val_acc, training_time



In [ ]:
def evaluate_model(model, test_loader, tumor_types):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    all_preds = []
    all_labels = []
    all_probs = []
    total_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = (all_preds == all_labels).mean()
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    auc_roc = roc_auc_score(all_labels, all_probs, average='weighted', multi_class='ovr')
    avg_loss = total_loss / len(test_loader.dataset)

    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')
    print(f'Average Loss: {avg_loss:.4f}')

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=tumor_types, yticklabels=tumor_types)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return accuracy, precision, recall, f1, auc_roc, avg_loss, all_preds, all_labels

def visualize_model_attention(model, input_tensor, target_class):
    model.eval()
    cam = GradCAM(model=model, target_layers=[model.layer4[-1]], use_cuda=torch.cuda.is_available())
    grayscale_cam = cam(input_tensor=input_tensor.unsqueeze(0), target_category=target_class)
    visualization = show_cam_on_image(input_tensor.permute(1, 2, 0).numpy(), grayscale_cam[0, :], use_rgb=True)
    plt.imshow(visualization)
    plt.axis('off')
    plt.title(f'Grad-CAM for class {target_class}')
    plt.show()

In [ ]:
def get_model_size(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p") / 1e6  # Size in MB
    os.remove('temp.p')
    return size

def create_metrics_dataframe(model, test_acc, precision, recall, f1, auc_roc, train_time, test_loss):
    metrics = {
        'Metric': ['Overall Accuracy', 'F1 Score', 'Cross Entropy Loss', 'Training Time (s)', 'Number of Parameters', 'Model Size (MB)'],
        'Value': [
            test_acc,
            f1,
            test_loss,
            train_time,
            sum(p.numel() for p in model.parameters()),
            get_model_size(model)
        ]
    }
    df = pd.DataFrame(metrics)
    return df

In [ ]:
# Define the base path in your Google Drive
base_path = '/content/drive/MyDrive/Colab Notebooks'

# Function to save the model
def save_model(model, filename):
    save_path = os.path.join(base_path, filename)
    try:
        torch.save(model.state_dict(), save_path)
        print(f"Model saved successfully to {save_path}")
    except Exception as e:
        print(f"Error saving model: {e}")

# Function to load the model
def load_model(model, filename):
    load_path = os.path.join(base_path, filename)
    try:
        model.load_state_dict(torch.load(load_path))
        print(f"Model loaded successfully from {load_path}")
    except Exception as e:
        print(f"Error loading model: {e}")
    return model


Training Loop: GPU T4

In [ ]:
# Main execution
if __name__ == "__main__":
    # Setup
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # Create the dataset
    full_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])
    results = []
    # K-Fold Cross-validation
    k_folds = 5
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(full_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(full_dataset, batch_size=32, sampler=train_subsampler)
        val_loader = DataLoader(full_dataset, batch_size=32, sampler=val_subsampler)

        model = initialize_model(num_classes)
        model, val_acc, train_time = train_model(model, train_loader, val_loader)

        results.append({
            'Fold': fold+1,
            'Validation Accuracy': val_acc,
            'Training Time (s)': train_time
        })

        # Save the model for this fold
        save_model(model, f'model_fold_{fold+1}.pth')

    # After k-fold cross-validation, train on the entire training set
    print('FINAL TRAINING')
    print('--------------------------------')
    train_loader = DataLoader(full_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    final_model = initialize_model(num_classes)
    final_model, final_val_acc, final_train_time = train_model(final_model, train_loader, val_loader)

    results.append({
        'Fold': 'Final',
        'Validation Accuracy': final_val_acc,
        'Training Time (s)': final_train_time
    })

    # Create and display the summary table
    summary_df = pd.DataFrame(results)
    print("\nTraining Summary:")
    print(summary_df.to_string(index=False))

    # Evaluate on the test set
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_acc, precision, recall, f1, auc_roc, test_loss, _, _ = evaluate_model(final_model, test_loader, tumor_types)

    print(f"\nFinal Test Accuracy: {test_acc:.4f}")

    # Create and display the metrics DataFrame
    metrics_df = create_metrics_dataframe(final_model, test_acc, precision, recall, f1, auc_roc, final_train_time, test_loss)
    print("\nModel Metrics:")
    print(metrics_df.to_string(index=False))

    # Save the DataFrame
    metrics_csv_path = os.path.join(base_path, 'model_metrics.csv')
    metrics_df.to_csv(metrics_csv_path, index=False)
    print(f"\nMetrics saved to {metrics_csv_path}")

    # Save the final model
    save_model(final_model, 'final_mri_classification_model.pth')

    print("Training, evaluation, and metrics logging complete!")

### GradCam for ResNet:
Load the model:

In [ ]:
resnet_model = initialize_model(num_classes=len(tumor_types))
resnet_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/final_mri_classification_model.pth'))
resnet_model.eval()

Grad-Cam:

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from torchvision import transforms
from PIL import Image
import os
from collections import defaultdict

def get_gradcam_visualization(model, image_path, training_order, target_layer_name='layer4'):
    """
    Generate Grad-CAM visualization using the correct class order from training
    """
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)

        # Use the same transforms as training
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # Display names mapping
        display_names = {
            'glioma': 'Glioma',
            'meningioma': 'Meningioma',
            'notumor': 'Normal',
            'pituitary': 'Pituitary'
        }

        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Get target layer
        target_layer = model.layer4[-1]

        grad_cam = GradCAM(
            model=model,
            target_layers=[target_layer],
            reshape_transform=None
        )

        # Get model prediction and confidence
        with torch.no_grad():
            output = model(input_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            prediction = torch.argmax(output).item()
            confidence = probabilities[0][prediction].item() * 100

            predicted_class = training_order[prediction]
            display_class = display_names[predicted_class]

        targets = [ClassifierOutputTarget(prediction)]
        grayscale_cam = grad_cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]

        rgb_img = np.array(image.resize((224, 224))) / 255.0
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

        return visualization, display_class, rgb_img, confidence, probabilities[0].cpu().numpy()

    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None, None, None, None, None

def visualize_correct_predictions(model, test_folder, training_order, samples_per_class=3):
    """
    Visualize Grad-CAM for correctly predicted images using original training order
    """
    display_names = {
        'glioma': 'Glioma',
        'meningioma': 'Meningioma',
        'notumor': 'Normal',
        'pituitary': 'Pituitary'
    }

    try:
        # Store correct predictions for each class
        correct_predictions = defaultdict(list)

        # First pass: collect correct predictions
        for class_name in training_order:
            folder_path = os.path.join(test_folder, class_name)

            if not os.path.exists(folder_path):
                print(f"Skipping {class_name} - folder not found: {folder_path}")
                continue

            print(f"\nProcessing {display_names[class_name]}...")
            image_files = [f for f in os.listdir(folder_path)
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            # Process all images in the folder
            for img_file in image_files:
                img_path = os.path.join(folder_path, img_file)
                vis, pred_class, original, confidence, probs = get_gradcam_visualization(
                    model, img_path, training_order)

                # Store correct predictions
                if vis is not None and pred_class == display_names[class_name]:
                    correct_predictions[class_name].append({
                        'path': img_path,
                        'visualization': vis,
                        'original': original,
                        'confidence': confidence,
                        'probabilities': probs
                    })

        # Create visualization for correct predictions
        plt.figure(figsize=(15, 3 * len(training_order)))

        for i, class_name in enumerate(training_order):
            predictions = correct_predictions[class_name]

            # Sort by confidence and take top k
            predictions.sort(key=lambda x: x['confidence'], reverse=True)
            selected_predictions = predictions[:samples_per_class]

            print(f"\nShowing top {len(selected_predictions)} correct predictions for {display_names[class_name]}")
            print(f"Total correct predictions: {len(predictions)}")

            for j, pred in enumerate(selected_predictions):
                plt.subplot(len(training_order), samples_per_class * 2, i * samples_per_class * 2 + j * 2 + 1)
                plt.imshow(pred['original'])
                plt.title(f'Original\n{display_names[class_name]}\nConf: {pred["confidence"]:.1f}%')
                plt.axis('off')

                plt.subplot(len(training_order), samples_per_class * 2, i * samples_per_class * 2 + j * 2 + 2)
                plt.imshow(pred['visualization'])
                plt.title('Grad-CAM')
                plt.axis('off')

                # Print probabilities for each class
                print(f"\nProbabilities for {display_names[class_name]} image {j+1}:")
                for cls, prob in zip(training_order, pred['probabilities']):
                    print(f"{display_names[cls]}: {prob*100:.2f}%")

        plt.tight_layout()
        plt.show()

        # Print summary statistics
        print("\nSummary of correct predictions:")
        for class_name in training_order:
            total = len(correct_predictions[class_name])
            print(f"{display_names[class_name]}: {total} correct predictions")

    except Exception as e:
        print(f"Error during visualization: {str(e)}")

# Example usage
training_order = ['glioma', 'meningioma', 'notumor', 'pituitary']

print("Generating Grad-CAM visualizations for correctly classified images...")
visualize_correct_predictions(
    model=resnet_model,
    test_folder=test_folder,
    training_order=training_order,
    samples_per_class=3
)

# Model 2: EfficientNetB0

In [ ]:
def initialize_model(num_classes):
    model = models.efficientnet_b0(pretrained=True)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    return model

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=100, patience=10):
    start_time = time.time()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

    best_val_loss = float('inf')
    best_val_acc = 0.0
    epochs_no_improve = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
             best_val_loss = val_loss
             best_val_acc = val_acc
             epochs_no_improve = 0
             best_model = model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            model.load_state_dict(best_model)
            break

    end_time = time.time()
    training_time = end_time - start_time
    return model, best_val_acc, training_time


In [ ]:
def visualize_model_attention(model, input_tensor, target_class):
    model.eval()
    cam = GradCAM(model=model, target_layers=[model.features[-1]], use_cuda=torch.cuda.is_available())
    grayscale_cam = cam(input_tensor=input_tensor.unsqueeze(0), target_category=target_class)
    visualization = show_cam_on_image(input_tensor.permute(1, 2, 0).numpy(), grayscale_cam[0, :], use_rgb=True)
    plt.imshow(visualization)
    plt.axis('off')
    plt.title(f'Grad-CAM for class {target_class}')
    plt.show()

In [ ]:
# Main execution
if __name__ == "__main__":
    # Setup
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # Create the dataset
    full_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])
    results = []

    # K-Fold Cross-validation
    k_folds = 5
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(full_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(full_dataset, batch_size=32, sampler=train_subsampler)
        val_loader = DataLoader(full_dataset, batch_size=32, sampler=val_subsampler)

        model = initialize_model(num_classes)
        model, val_acc, train_time = train_model(model, train_loader, val_loader)

        results.append({
            'Fold': fold+1,
            'Validation Accuracy': val_acc,
            'Training Time (s)': train_time
        })

        # Save the model for this fold
        save_model(model, f'efficientnet_b0_model_fold_{fold+1}.pth')

    # After k-fold cross-validation, train on the entire training set
    print('FINAL TRAINING')
    print('--------------------------------')
    train_loader = DataLoader(full_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    final_model = initialize_model(num_classes)
    final_model, final_val_acc, final_train_time = train_model(final_model, train_loader, val_loader)

    results.append({
        'Fold': 'Final',
        'Validation Accuracy': final_val_acc,
        'Training Time (s)': final_train_time
    })

    # Create and display the summary table
    summary_df = pd.DataFrame(results)
    print("\nTraining Summary:")
    print(summary_df.to_string(index=False))

    # Evaluate on the test set
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_acc, precision, recall, f1, auc_roc, test_loss, _, _ = evaluate_model(final_model, test_loader, tumor_types)

    print(f"\nFinal Test Accuracy: {test_acc:.4f}")

    # Create and display the metrics DataFrame
    metrics_df = create_metrics_dataframe(final_model, test_acc, precision, recall, f1, auc_roc, final_train_time, test_loss)
    print("\nModel Metrics:")
    print(metrics_df.to_string(index=False))

    # Save the DataFrame
    metrics_csv_path = os.path.join(base_path, 'efficientnet_b0_model_metrics.csv')
    metrics_df.to_csv(metrics_csv_path, index=False)
    print(f"\nMetrics saved to {metrics_csv_path}")

    # Save the final model
    save_model(final_model, 'final_efficientnet_b0_mri_classification_model.pth')

    print("Training, evaluation, and metrics logging complete for EfficientNet-B0 model!")

### Grad-Cam for EfficientNet
Load the model:

In [ ]:
efficientnet_model = initialize_model(num_classes=len(tumor_types))
efficientnet_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/final_efficientnet_b0_mri_classification_model.pth'))
efficientnet_model.eval()

Grad-cam:

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from torchvision import transforms
from PIL import Image
import os
from collections import defaultdict

def get_gradcam_visualization(model, image_path, training_order, target_layer_name='features.8'):
    """
    Generate Grad-CAM visualization for EfficientNetB0 using the correct class order from training
    """
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)

        # Use the same transforms as training
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # Display names mapping
        display_names = {
            'glioma': 'Glioma',
            'meningioma': 'Meningioma',
            'notumor': 'Normal',
            'pituitary': 'Pituitary'
        }

        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Get target layer - for EfficientNetB0, we use the final MBConv block
        target_layer = model.features[8]

        grad_cam = GradCAM(
            model=model,
            target_layers=[target_layer],
            reshape_transform=None
        )

        # Get model prediction and confidence
        with torch.no_grad():
            output = model(input_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            prediction = torch.argmax(output).item()
            confidence = probabilities[0][prediction].item() * 100

            predicted_class = training_order[prediction]
            display_class = display_names[predicted_class]

        targets = [ClassifierOutputTarget(prediction)]
        grayscale_cam = grad_cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]

        rgb_img = np.array(image.resize((224, 224))) / 255.0
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

        return visualization, display_class, rgb_img, confidence, probabilities[0].cpu().numpy()

    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None, None, None, None, None

def visualize_correct_predictions(model, test_folder, training_order, samples_per_class=3):
    """
    Visualize Grad-CAM for correctly predicted images using original training order
    """
    display_names = {
        'glioma': 'Glioma',
        'meningioma': 'Meningioma',
        'notumor': 'Normal',
        'pituitary': 'Pituitary'
    }

    try:
        # Store correct predictions for each class
        correct_predictions = defaultdict(list)

        # First pass: collect correct predictions
        for class_name in training_order:
            folder_path = os.path.join(test_folder, class_name)

            if not os.path.exists(folder_path):
                print(f"Skipping {class_name} - folder not found: {folder_path}")
                continue

            print(f"\nProcessing {display_names[class_name]}...")
            image_files = [f for f in os.listdir(folder_path)
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            # Process all images in the folder
            for img_file in image_files:
                img_path = os.path.join(folder_path, img_file)
                vis, pred_class, original, confidence, probs = get_gradcam_visualization(
                    model, img_path, training_order)

                # Store correct predictions
                if vis is not None and pred_class == display_names[class_name]:
                    correct_predictions[class_name].append({
                        'path': img_path,
                        'visualization': vis,
                        'original': original,
                        'confidence': confidence,
                        'probabilities': probs
                    })

        # Create visualization for correct predictions
        plt.figure(figsize=(15, 3 * len(training_order)))

        for i, class_name in enumerate(training_order):
            predictions = correct_predictions[class_name]

            # Sort by confidence and take top k
            predictions.sort(key=lambda x: x['confidence'], reverse=True)
            selected_predictions = predictions[:samples_per_class]

            print(f"\nShowing top {len(selected_predictions)} correct predictions for {display_names[class_name]}")
            print(f"Total correct predictions: {len(predictions)}")

            for j, pred in enumerate(selected_predictions):
                plt.subplot(len(training_order), samples_per_class * 2, i * samples_per_class * 2 + j * 2 + 1)
                plt.imshow(pred['original'])
                plt.title(f'Original\n{display_names[class_name]}\nConf: {pred["confidence"]:.1f}%')
                plt.axis('off')

                plt.subplot(len(training_order), samples_per_class * 2, i * samples_per_class * 2 + j * 2 + 2)
                plt.imshow(pred['visualization'])
                plt.title('Grad-CAM')
                plt.axis('off')

                # Print probabilities for each class
                print(f"\nProbabilities for {display_names[class_name]} image {j+1}:")
                for cls, prob in zip(training_order, pred['probabilities']):
                    print(f"{display_names[cls]}: {prob*100:.2f}%")

        plt.tight_layout()
        plt.show()

        # Print summary statistics
        print("\nSummary of correct predictions:")
        for class_name in training_order:
            total = len(correct_predictions[class_name])
            print(f"{display_names[class_name]}: {total} correct predictions")

    except Exception as e:
        print(f"Error during visualization: {str(e)}")

# Example usage
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

print("Generating Grad-CAM visualizations for correctly classified images...")
visualize_correct_predictions(
    model=efficientnet_model,
    test_folder=test_folder,
    training_order=tumor_types,
    samples_per_class=3
)

##Remark:
EfficienNet and ResNet are both CNNs that specialize in edge detection by nature. So it is normal to observe that model attention goes to the edge of tumours instead of directly hovering above it.

## Comparaison of ResNet50 and EfficientNetB0:

In [ ]:
# Assuming the data is already in the format you provided
data = {
    'EfficientNet_Metric': ['Overall Accuracy', 'F1 Score', 'Cross Entropy Loss', 'Training Time (s)', 'Number of Parameters', 'Model Size (MB)'],
    'EfficientNet_Value': [9.847445e-01, 9.846879e-01, 5.018997e-02, 2.509030e+03, 4.012672e+06, 1.632957e+01],
    'OtherModel_Metric': ['Overall Accuracy', 'F1 Score', 'Cross Entropy Loss', 'Training Time (s)', 'Number of Parameters', 'Model Size (MB)'],
    'OtherModel_Value': [9.938978e-01, 9.938897e-01, 2.039252e-02, 2.520090e+03, 2.351623e+07, 9.436445e+01]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Create separate bar charts for each metric
metrics = df['EfficientNet_Metric']
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Comparison of Model Metrics', fontsize=16)

for i, metric in enumerate(metrics):
    ax = axes[i // 3, i % 3]

    efficientnet_value = df.loc[df['EfficientNet_Metric'] == metric, 'EfficientNet_Value'].values[0]
    othermodel_value = df.loc[df['OtherModel_Metric'] == metric, 'OtherModel_Value'].values[0]

    ax.bar(['EfficientNet', 'Other Model'], [efficientnet_value, othermodel_value])
    ax.set_title(metric)
    ax.set_ylabel('Value')

    # Format y-axis labels based on the metric
    if 'Parameters' in metric or 'Time' in metric:
        ax.set_yscale('log')
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    elif 'Accuracy' in metric or 'F1 Score' in metric:
        ax.set_ylim(0.95, 1.0)  # Adjust as needed

    # Add value labels on top of each bar
    for j, v in enumerate([efficientnet_value, othermodel_value]):
        ax.text(j, v, f'{v:.2e}', ha='center', va='bottom')

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

# Create a table to show the exact values
fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('off')
table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)
plt.title('Model Metrics Comparison Table')
plt.tight_layout()
plt.show()



__It make sense to use EfficientNetB0, because it has a lot less parameters than ResNet50, yet achieves almost the same accuracy.__

# Model 3: Levit-256

In [ ]:
import os
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import KFold
import timm
import time
import warnings

In [ ]:
class LeViT256Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(LeViT256Model, self).__init__()

        # Load pretrained LeViT-256 model
        self.levit = timm.create_model('levit_256', pretrained=pretrained, num_classes=0)

        # Get the number of features from LeViT
        levit_num_features = self.levit.num_features

        # Add final classification layers
        self.fc = nn.Sequential(
            nn.Linear(levit_num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Process input through LeViT
        features = self.levit(x)

        # Final classification
        output = self.fc(features)
        return output

# Function to initialize the model
def initialize_levit_model(num_classes, pretrained=True):
    return LeViT256Model(num_classes, pretrained)

# Evaluation Function
def evaluate_model(model, test_loader, tumor_types):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    all_preds = []
    all_labels = []
    all_probs = []
    total_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = (all_preds == all_labels).mean()
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    auc_roc = roc_auc_score(all_labels, all_probs, average='weighted', multi_class='ovr')
    avg_loss = total_loss / len(test_loader.dataset)

    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')
    print(f'Average Loss: {avg_loss:.4f}')

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=tumor_types, yticklabels=tumor_types)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return accuracy, precision, recall, f1, auc_roc, avg_loss, all_preds, all_labels

# Function to create metrics DataFrame
def create_metrics_dataframe(model, test_acc, precision, recall, f1, auc_roc, train_time, test_loss):
    metrics = {
        'Metric': ['Overall Accuracy', 'F1 Score', 'Cross Entropy Loss', 'Training Time (s)', 'Number of Parameters', 'Model Size (MB)'],
        'Value': [
            test_acc,
            f1,
            test_loss,
            train_time,
            sum(p.numel() for p in model.parameters()),
            sum(p.nelement() * p.element_size() for p in model.parameters()) / (1024 * 1024)
        ]
    }
    df = pd.DataFrame(metrics)
    return df

In [ ]:
def train_levit_model(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    criterion = nn.CrossEntropyLoss()

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_subsampler, num_workers=2, pin_memory=True)
        val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_subsampler, num_workers=2, pin_memory=True)

        model = initialize_levit_model(num_classes).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        best_val_loss = float('inf')
        best_model = None
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            # Training phase
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            # Calculate average losses and accuracies
            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total

            print(f'Epoch {epoch+1}/{num_epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            # Save the first model state or if we have a new best validation loss
            if best_model is None or val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Final training on entire dataset
    print('FINAL TRAINING')
    print('--------------------------------')
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    final_model = initialize_levit_model(num_classes).to(device)
    optimizer = optim.Adam(final_model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    best_val_loss = float('inf')
    best_model = None  # Initialize best_model
    epochs_no_improve = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        final_model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = final_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            except RuntimeError as e:
                print(f"RuntimeError in final training loop: {e}")
                print(f"Input shape: {inputs.shape}")
                continue

        # Validation phase
        final_model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = final_model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
                except RuntimeError as e:
                    print(f"RuntimeError in final validation loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

        # Calculate average losses and accuracies
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(test_loader.dataset)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        scheduler.step(val_loss)

        # Save the first model state or if we have a new best validation loss
        if best_model is None or val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = final_model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            break

    end_time = time.time()
    final_training_time = end_time - start_time
    results.append({
        'Fold': 'Final',
        'Best Validation Loss': best_val_loss,
        'Training Time (s)': final_training_time
    })

    # Load the best model state
    final_model.load_state_dict(best_model)

    # Save the final model
    torch.save({
        'epoch': len(results),
        'model_state_dict': best_model,
        'results': results
    }, 'final_levit_256_model.pth')

    return results, final_model

In [ ]:
warnings.filterwarnings('ignore')

# Disable torch compile to avoid dynamo issues
torch._dynamo.config.suppress_errors = True

if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    try:
        # Create the datasets
        train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
        test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

        # Print dataset sizes
        print("Dataset sizes:")
        print(f"Training: {len(train_dataset)}")
        print(f"Testing: {len(test_dataset)}")

        # Train the LeViT-256 model
        results, final_model = train_levit_model(train_dataset, test_dataset, num_classes)

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'levit_256_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Evaluate on the test set
        test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display the metrics DataFrame
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save the metrics DataFrame
        metrics_csv_path = os.path.join(base_path, 'levit_256_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save the final model with all relevant information
        model_path = os.path.join(base_path, 'levit_256_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'levit_256_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()  # This will print the full error traceback
        raise e

    print("\nLeViT-256 training, evaluation, and metrics logging complete!")

## New GradCam Version

In [ ]:
levit256_model = initialize_levit_model(num_classes=4)
levit256_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/levit_256_model_final.pth')['model_state_dict'])
levit256_model = levit256_model.cuda()
levit256_model.eval()

## Old GradCam Version

In [ ]:
import cv2
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

def visualize_gradcam(model, image_path, transform, tumor_types, target_class=None, alpha=0.5):
    """
    Visualize Grad-CAM for a given image and model.

    Args:
        model: The trained LeViT model
        image_path: Path to the image file
        transform: Transformation pipeline
        tumor_types: List of tumor type labels
        target_class: Specific class to visualize (optional)
        alpha: Transparency of the heatmap overlay
    """
    # Set model to eval mode
    model.eval()

    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)
    input_tensor = input_tensor.to(next(model.parameters()).device)

    # Get the target layer (last convolutional layer in LeViT)
    target_layer = model.levit.stages[-1][-1].norm  # Adjust this based on LeViT architecture

    # Initialize GradCAM
    grad_cam = GradCAM(model, target_layer)

    # Get model prediction
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output, dim=1)
        predicted_class = output.argmax(dim=1).item()

    # If target class is not specified, use predicted class
    if target_class is None:
        target_class = predicted_class

    # Generate heatmap
    heatmap = grad_cam.get_cam(input_tensor, target_class)

    # Convert image to numpy array
    orig_img = np.array(img)

    # Resize heatmap to match image size
    heatmap = cv2.resize(heatmap, (orig_img.shape[1], orig_img.shape[0]))

    # Convert heatmap to RGB
    heatmap_rgb = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_rgb = cv2.cvtColor(heatmap_rgb, cv2.COLOR_BGR2RGB)

    # Create superimposed image
    superimposed = cv2.addWeighted(orig_img, 1-alpha, heatmap_rgb, alpha, 0)

    # Plot the results
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    # Original image
    ax[0].imshow(orig_img)
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    # Heatmap
    ax[1].imshow(heatmap_rgb)
    ax[1].set_title('Grad-CAM Heatmap')
    ax[1].axis('off')

    # Superimposed
    ax[2].imshow(superimposed)
    ax[2].set_title('Superimposed')
    ax[2].axis('off')

    # Add prediction information
    pred_prob = probabilities[0][predicted_class].item() * 100
    fig.suptitle(f'Prediction: {tumor_types[predicted_class]} ({pred_prob:.2f}%)', fontsize=14)

    plt.tight_layout()
    plt.show()

    return heatmap, predicted_class, probabilities[0].cpu().numpy()

def visualize_batch_gradcam(model, image_paths, transform, tumor_types, output_dir=None):
    """
    Generate Grad-CAM visualizations for a batch of images.

    Args:
        model: The trained LeViT model
        image_paths: List of paths to images
        transform: Transformation pipeline
        tumor_types: List of tumor type labels
        output_dir: Directory to save visualizations (optional)
    """
    for idx, img_path in enumerate(image_paths):
        print(f"\nProcessing image {idx+1}/{len(image_paths)}")
        print(f"Image path: {img_path}")

        heatmap, pred_class, probs = visualize_gradcam(model, img_path, transform, tumor_types)

        # Print prediction probabilities for each class
        print("\nPrediction probabilities:")
        for i, (tumor_type, prob) in enumerate(zip(tumor_types, probs)):
            print(f"{tumor_type}: {prob*100:.2f}%")

        if output_dir:
            plt.savefig(os.path.join(output_dir, f'gradcam_{idx}.png'))

        plt.close()

# Example usage function
def analyze_model_attention(model, test_dataset, tumor_types, num_samples=5):
    """
    Analyze model attention on random test samples.

    Args:
        model: Trained LeViT model
        test_dataset: Test dataset
        tumor_types: List of tumor type labels
        num_samples: Number of random samples to analyze
    """
    # Get random indices
    indices = np.random.choice(len(test_dataset), num_samples, replace=False)

    print(f"\nAnalyzing model attention on {num_samples} random test samples...")

    for idx in indices:
        # Get image path and true label
        img_path = test_dataset.image_paths[idx]
        true_label = test_dataset.labels[idx]

        print(f"\nAnalyzing image {idx}")
        print(f"True label: {tumor_types[true_label]}")

        # Generate Grad-CAM visualization
        heatmap, pred_class, probs = visualize_gradcam(
            model,
            img_path,
            test_dataset.transform,
            tumor_types
        )

        print(f"Predicted label: {tumor_types[pred_class]}")
        print("\nClass probabilities:")
        for tumor_type, prob in zip(tumor_types, probs):
            print(f"{tumor_type}: {prob*100:.2f}%")

In [ ]:
class GradCAM:
    def __init__(self, model):
        self.model = model
        self.target_layer = model.levit.stages[-1].blocks[-1].attn
        self.gradients = None
        self.activation = None

        def forward_hook(module, input, output):
            self.activation = output.detach()
            return output

        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def generate_cam(self, input_tensor, target_class=None):
        b, c, h, w = input_tensor.shape

        # Forward pass
        with torch.set_grad_enabled(True):
            model_output = self.model(input_tensor)

            if target_class is None:
                target_class = model_output.argmax(dim=1).item()

            # Zero grads
            self.model.zero_grad()

            # Target for backprop
            one_hot = torch.zeros_like(model_output)
            one_hot[0][target_class] = 1

            # Backward pass with retain_graph
            model_output.backward(gradient=one_hot, retain_graph=True)

        # Generate attention map
        if self.gradients is None or self.activation is None:
            return np.zeros((h, w)), model_output

        # Get the attention map
        attention_map = self.activation.detach()  # Shape: [B, N, D]

        # Reshape attention map to 2D
        attention_size = int(np.sqrt(attention_map.size(1)))
        attention_map = attention_map.mean(dim=2)  # Average over features
        attention_map = attention_map.reshape(1, 1, attention_size, attention_size)

        # Interpolate to input size
        with torch.no_grad():
            attention_map = F.interpolate(attention_map,
                                        size=(h, w),
                                        mode='bilinear',
                                        align_corners=False)

        # Normalize
        attention_map = attention_map - attention_map.min()
        attention_map = attention_map / (attention_map.max() + 1e-8)

        # Detach and convert to numpy
        attention_map = attention_map.squeeze().detach().cpu().numpy()

        return attention_map, model_output.detach()

def visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3):
    """
    Visualize attention maps for specific samples from each class
    """
    model.eval()
    device = next(model.parameters()).device
    grad_cam = GradCAM(model)

    # Organize samples by class
    samples_by_class = {}
    for idx in range(len(test_dataset)):
        _, label = test_dataset[idx]
        if label not in samples_by_class:
            samples_by_class[label] = []
        samples_by_class[label].append(idx)

    # Process samples from each class
    for class_idx in range(len(tumor_types)):
        print(f"\nProcessing class: {tumor_types[class_idx]}")

        # Get random samples for this class
        class_samples = random.sample(samples_by_class[class_idx], min(samples_per_class, len(samples_by_class[class_idx])))

        for sample_idx in class_samples:
            try:
                # Get image and create input tensor
                image, label = test_dataset[sample_idx]
                input_tensor = image.unsqueeze(0).to(device)

                # Generate attention map
                with torch.no_grad():
                    attention_map, output = grad_cam.generate_cam(input_tensor)
                    probs = F.softmax(output, dim=1)
                    predicted_class = output.argmax(dim=1).item()

                # Get original image
                original_image = cv2.imread(test_dataset.image_paths[sample_idx])
                original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

                # Create heatmap
                heatmap = np.uint8(255 * attention_map)
                heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))
                heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

                # Superimpose heatmap on original image
                superimposed = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0)

                # Create figure with subplots
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))

                # Plot original image
                axes[0].imshow(original_image)
                axes[0].set_title('Original Image')
                axes[0].axis('off')

                # Plot heatmap
                axes[1].imshow(cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB))
                axes[1].set_title('Attention Map')
                axes[1].axis('off')

                # Plot superimposed image
                axes[2].imshow(superimposed)
                axes[2].set_title('Superimposed')
                axes[2].axis('off')

                # Add prediction information
                true_class = tumor_types[label]
                pred_class = tumor_types[predicted_class]
                pred_prob = probs[0][predicted_class].item() * 100

                plt.suptitle(f'True: {true_class} | Predicted: {pred_class} ({pred_prob:.2f}%)\n'
                            f'Image: {test_dataset.image_paths[sample_idx].split("/")[-1]}',
                            fontsize=12)

                plt.tight_layout()
                plt.show()
                plt.close()

            except Exception as e:
                print(f"Error processing image at index {sample_idx}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

In [ ]:
def create_smooth_heatmap(attention_map, original_image, colormap=cv2.COLORMAP_INFERNO, alpha=0.4):
    """
    Create a smooth, visually appealing heatmap overlay

    Args:
        attention_map: The attention map from Grad-CAM
        original_image: Original input image
        colormap: OpenCV colormap to use (default: COLORMAP_INFERNO)
        alpha: Transparency of the heatmap overlay (default: 0.4)
    """
    # Apply Gaussian blur to smooth the attention map
    attention_map = cv2.GaussianBlur(attention_map, (3, 3), 0)

    # Resize attention map to match original image size
    heatmap = cv2.resize(attention_map, (original_image.shape[1], original_image.shape[0]))

    # Normalize to 0-255 range
    heatmap = np.uint8(255 * heatmap)

    # Apply colormap
    heatmap = cv2.applyColorMap(heatmap, colormap)

    # Convert to RGB
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Create superimposed image with smoother blending
    superimposed = cv2.addWeighted(original_image, 1 - alpha, heatmap, alpha, 0)

    # Enhance contrast of the superimposed image
    superimposed = cv2.convertScaleAbs(superimposed, alpha=1.2, beta=10)

    return heatmap, superimposed

def visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3):
    """
    Visualize attention maps for specific samples from each class
    """
    model.eval()
    device = next(model.parameters()).device
    grad_cam = GradCAM(model)

    # Process samples from each class
    for class_idx in range(len(tumor_types)):
        print(f"\nProcessing class: {tumor_types[class_idx]}")

        # Get samples for this class
        class_samples = [idx for idx, (_, label) in enumerate(test_dataset) if label == class_idx]
        selected_samples = random.sample(class_samples, min(samples_per_class, len(class_samples)))

        for sample_idx in selected_samples:
            try:
                # Get image and create input tensor
                image, label = test_dataset[sample_idx]
                input_tensor = image.unsqueeze(0).to(device)

                # Generate attention map
                with torch.no_grad():
                    attention_map, output = grad_cam.generate_cam(input_tensor)
                    probs = F.softmax(output, dim=1)
                    predicted_class = output.argmax(dim=1).item()

                # Get original image
                original_image = cv2.imread(test_dataset.image_paths[sample_idx])
                original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

                # Create smooth heatmap visualization
                heatmap, superimposed = create_smooth_heatmap(
                    attention_map,
                    original_image,
                    colormap=cv2.COLORMAP_MAGMA,  # Using MAGMA colormap for better visualization
                    alpha=0.5
                )

                # Create figure with subplots
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))

                # Plot original image
                axes[0].imshow(original_image)
                axes[0].set_title('Original Image', fontsize=12)
                axes[0].axis('off')

                # Plot heatmap
                axes[1].imshow(heatmap)
                axes[1].set_title('Attention Map', fontsize=12)
                axes[1].axis('off')

                # Plot superimposed image
                axes[2].imshow(superimposed)
                axes[2].set_title('Superimposed', fontsize=12)
                axes[2].axis('off')

                # Add prediction information
                true_class = tumor_types[label]
                pred_class = tumor_types[predicted_class]
                pred_prob = probs[0][predicted_class].item() * 100

                plt.suptitle(
                    f'True: {true_class} | Predicted: {pred_class} ({pred_prob:.2f}%)\n'
                    f'Image: {test_dataset.image_paths[sample_idx].split("/")[-1]}',
                    fontsize=13,
                    y=1.02
                )

                # Adjust layout
                plt.tight_layout()

                # Add a white background
                fig.patch.set_facecolor('white')

                plt.show()
                plt.close()

            except Exception as e:
                print(f"Error processing image at index {sample_idx}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

In [ ]:
def create_refined_heatmap(attention_map, original_image):
    """
    Create a refined heatmap visualization with better color mapping
    """
    # Smooth the attention map
    attention_map = cv2.GaussianBlur(attention_map, (3, 3), 0)

    # Resize to match original image
    heatmap = cv2.resize(attention_map, (original_image.shape[1], original_image.shape[0]))

    # Create colormap (red -> yellow -> green -> blue)
    colors = []
    for i in range(256):
        if i < 64:  # blue to cyan
            colors.append([0, i*4, 255])
        elif i < 128:  # cyan to green
            j = i - 64
            colors.append([0, 255, 255 - j*4])
        elif i < 192:  # green to yellow
            j = i - 128
            colors.append([j*4, 255, 0])
        else:  # yellow to red
            j = i - 192
            colors.append([255, 255 - j*4, 0])

    colors = np.array(colors, dtype=np.uint8)

    # Apply colormap
    heatmap_normalized = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap_normalized, cv2.COLORMAP_JET)
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)

    # Create superimposed image with better blending
    superimposed = np.float32(original_image.copy())

    # Enhanced alpha blending
    alpha = 0.6
    mask = heatmap > 0.2  # Only show significant activations
    superimposed[mask] = superimposed[mask] * (1 - alpha) + heatmap_colored[mask] * alpha

    return heatmap_normalized, np.uint8(superimposed)

def visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3):
    model.eval()
    device = next(model.parameters()).device
    grad_cam = GradCAM(model)

    plt.style.use('default')

    for class_idx in range(len(tumor_types)):
        print(f"\nProcessing class: {tumor_types[class_idx]}")

        class_samples = [idx for idx, (_, label) in enumerate(test_dataset) if label == class_idx]
        selected_samples = random.sample(class_samples, min(samples_per_class, len(class_samples)))

        for sample_idx in selected_samples:
            try:
                # Get image and predict
                image, label = test_dataset[sample_idx]
                input_tensor = image.unsqueeze(0).to(device)

                # Generate attention map
                with torch.no_grad():
                    attention_map, output = grad_cam.generate_cam(input_tensor)
                    probs = F.softmax(output, dim=1)
                    predicted_class = output.argmax(dim=1).item()

                # Get original image
                original_image = cv2.imread(test_dataset.image_paths[sample_idx])
                original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

                # Create visualization
                heatmap, superimposed = create_refined_heatmap(attention_map, original_image)

                # Create figure
                fig, axes = plt.subplots(1, 3, figsize=(15, 4))
                fig.patch.set_facecolor('white')
                plt.subplots_adjust(wspace=0.3)

                # Plot original image
                axes[0].imshow(original_image, cmap='gray')
                axes[0].set_title('Original Image')
                axes[0].axis('off')

                # Plot attention map
                im = axes[1].imshow(heatmap, cmap='jet')
                axes[1].set_title('Attention Map')
                axes[1].axis('off')
                cbar = plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
                cbar.set_ticks([0, 64, 128, 192, 255])
                cbar.set_ticklabels(['0.0', '0.2', '0.4', '0.6', '0.8'])

                # Plot superimposed
                axes[2].imshow(superimposed)
                axes[2].set_title('CNN GradCAM')
                axes[2].axis('off')

                # Add prediction information
                true_class = tumor_types[label]
                pred_class = tumor_types[predicted_class]
                pred_prob = probs[0][predicted_class].item() * 100

                plt.suptitle(
                    f'True: {true_class}\nPred: {pred_class} ({pred_prob:.2f}%)',
                    y=1.05,
                    fontsize=12
                )

                plt.show()
                plt.close()

            except Exception as e:
                print(f"Error processing image at index {sample_idx}: {str(e)}")
                traceback.print_exc()
                continue

In [ ]:
# Main execution code
if __name__ == "__main__":
    # 1. Setup transforms
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # 2. Create dataset
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)

    # 3. Load model
    model = initialize_levit_model(num_classes=4)
    model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/levit_256_model_final.pth')['model_state_dict'])
    model = model.cuda()

    # 4. Visualize attention maps
    visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3)

# Model 4: Levit-384

In [ ]:
class LeViT384Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(LeViT384Model, self).__init__()

        # Load pretrained LeViT-384 model
        self.levit = timm.create_model('levit_384', pretrained=pretrained, num_classes=0)

        # Get the number of features from LeViT
        levit_num_features = self.levit.num_features

        # Add final classification layers
        self.fc = nn.Sequential(
            nn.Linear(levit_num_features, 1024),  # Increased intermediate layer size for LeViT-384
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        # Process input through LeViT
        features = self.levit(x)

        # Final classification
        output = self.fc(features)
        return output

# Function to initialize the model
def initialize_levit_model(num_classes, pretrained=True):
    return LeViT384Model(num_classes, pretrained)

In [ ]:
def train_levit_model(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    criterion = nn.CrossEntropyLoss()

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_subsampler, num_workers=2, pin_memory=True)
        val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_subsampler, num_workers=2, pin_memory=True)

        model = initialize_levit_model(num_classes).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        best_val_loss = float('inf')
        best_model = None
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            # Training phase
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            # Calculate average losses and accuracies
            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total

            print(f'Epoch {epoch+1}/{num_epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            # Save the first model state or if we have a new best validation loss
            if best_model is None or val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Final training on entire dataset
    print('FINAL TRAINING')
    print('--------------------------------')
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    final_model = initialize_levit_model(num_classes).to(device)
    optimizer = optim.Adam(final_model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    best_val_loss = float('inf')
    best_model = None  # Initialize best_model
    epochs_no_improve = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        final_model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = final_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            except RuntimeError as e:
                print(f"RuntimeError in final training loop: {e}")
                print(f"Input shape: {inputs.shape}")
                continue

        # Validation phase
        final_model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = final_model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
                except RuntimeError as e:
                    print(f"RuntimeError in final validation loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

        # Calculate average losses and accuracies
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(test_loader.dataset)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        scheduler.step(val_loss)

        # Save the first model state or if we have a new best validation loss
        if best_model is None or val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = final_model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            break

    end_time = time.time()
    final_training_time = end_time - start_time
    results.append({
        'Fold': 'Final',
        'Best Validation Loss': best_val_loss,
        'Training Time (s)': final_training_time
    })

    # Load the best model state
    final_model.load_state_dict(best_model)

    # Save the final model
    torch.save({
        'epoch': len(results),
        'model_state_dict': best_model,
        'results': results
    }, 'final_levit_384_model.pth')

    return results, final_model

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import KFold

warnings.filterwarnings('ignore')

# Disable torch compile to avoid dynamo issues
torch._dynamo.config.suppress_errors = True

if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    try:
        # Create the datasets
        train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
        test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

        # Print dataset sizes
        print("Dataset sizes:")
        print(f"Training: {len(train_dataset)}")
        print(f"Testing: {len(test_dataset)}")

        # Train the LeViT-384 model
        results, final_model = train_levit_model(train_dataset, test_dataset, num_classes)

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'levit_384_training_history.csv')  # Changed filename
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Evaluate on the test set
        # Reduced batch size for LeViT-384 due to higher memory requirements
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)  # Reduced batch size
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display the metrics DataFrame
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save the metrics DataFrame
        metrics_csv_path = os.path.join(base_path, 'levit_384_model_metrics.csv')  # Changed filename
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save the final model with all relevant information
        model_path = os.path.join(base_path, 'levit_384_model_final.pth')  # Changed filename
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'levit_384_test_predictions.csv')  # Changed filename
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()  # This will print the full error traceback
        raise e

    print("\nLeViT-384 training, evaluation, and metrics logging complete!")  # Updated message

In [ ]:
import traceback
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from torch.nn.functional import interpolate

class GradCAM:
    def __init__(self, model):
        self.model = model
        self.target_layer = model.levit.stages[-1].blocks[-1].mlp
        self.gradients = None
        self.activation = None

        def forward_hook(module, input, output):
            self.activation = output[0] if isinstance(output, tuple) else output
            self.activation = self.activation.detach()
            return output

        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def generate_cam(self, input_tensor, target_class=None):
        b, c, h, w = input_tensor.shape

        # Forward pass
        with torch.set_grad_enabled(True):
            model_output = self.model(input_tensor)

            if target_class is None:
                target_class = model_output.argmax(dim=1).item()

            self.model.zero_grad()
            one_hot = torch.zeros_like(model_output)
            one_hot[0][target_class] = 1
            model_output.backward(gradient=one_hot, retain_graph=True)

        if self.gradients is None or self.activation is None:
            return np.zeros((h, w)), model_output

        # Print shapes for debugging
        print(f"Activation shape: {self.activation.shape}")
        print(f"Gradients shape: {self.gradients.shape}")

        # Handle transformer-style output (B, N, C)
        if len(self.activation.shape) == 3:
            B, N, C = self.activation.shape

            # Calculate spatial dimensions
            H = W = int(np.sqrt(N))

            # Reshape activation and gradients to (B, C, H, W)
            activation = self.activation.permute(0, 2, 1).reshape(B, C, H, W)
            gradients = self.gradients.permute(0, 2, 1).reshape(B, C, H, W)

            # Calculate channel-wise weights
            weights = gradients.mean(dim=(2, 3), keepdim=True)

            # Weight the activation maps
            cam = (weights * activation).sum(dim=1, keepdim=True)

        else:
            # For regular convolutional output (B, C, H, W)
            weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
            cam = (weights * self.activation).sum(dim=1, keepdim=True)

        # Apply ReLU and normalize
        cam = F.relu(cam)

        # Resize to input size
        cam = F.interpolate(
            cam,
            size=(h, w),
            mode='bilinear',
            align_corners=False
        )

        # Normalize
        with torch.no_grad():
            cam_min = cam.min()
            cam_max = cam.max()
            if cam_max - cam_min != 0:
                cam = (cam - cam_min) / (cam_max - cam_min)

        return cam[0, 0].cpu().numpy(), model_output.detach()

def create_refined_heatmap(attention_map, original_image):
    """
    Create a refined heatmap visualization with better color mapping
    """
    # Ensure attention_map is a numpy array
    if torch.is_tensor(attention_map):
        attention_map = attention_map.cpu().numpy()

    # Ensure attention_map is 2D
    if len(attention_map.shape) > 2:
        attention_map = attention_map.squeeze()

    # Normalize if needed
    if attention_map.max() > 1:
        attention_map = attention_map / attention_map.max()

    # Smooth the attention map
    attention_map = cv2.GaussianBlur(attention_map.astype(np.float32), (3, 3), 0)

    # Resize to match original image
    heatmap = cv2.resize(attention_map, (original_image.shape[1], original_image.shape[0]))

    # Apply colormap
    heatmap_normalized = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap_normalized, cv2.COLORMAP_JET)
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)

    # Create superimposed image with better blending
    superimposed = np.float32(original_image.copy())
    alpha = 0.6
    mask = heatmap > 0.2
    superimposed[mask] = superimposed[mask] * (1 - alpha) + heatmap_colored[mask] * alpha

    return heatmap_normalized, np.uint8(superimposed)

def visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3):
    model.eval()
    device = next(model.parameters()).device
    grad_cam = GradCAM(model)

    plt.style.use('default')

    for class_idx in range(len(tumor_types)):
        print(f"\nProcessing class: {tumor_types[class_idx]}")

        class_samples = [idx for idx, (_, label) in enumerate(test_dataset) if label == class_idx]
        selected_samples = random.sample(class_samples, min(samples_per_class, len(class_samples)))

        for sample_idx in selected_samples:
            try:
                # Get image and predict
                image, label = test_dataset[sample_idx]
                input_tensor = image.unsqueeze(0).to(device)

                # Generate attention map
                attention_map, output = grad_cam.generate_cam(input_tensor)
                probs = F.softmax(output, dim=1)
                predicted_class = output.argmax(dim=1).item()

                # Get original image
                original_image = cv2.imread(test_dataset.image_paths[sample_idx])
                original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

                # Create visualization
                heatmap, superimposed = create_refined_heatmap(attention_map, original_image)

                # Create figure
                fig, axes = plt.subplots(1, 3, figsize=(15, 4))
                fig.patch.set_facecolor('white')
                plt.subplots_adjust(wspace=0.3)

                # Plot visualizations
                axes[0].imshow(original_image)
                axes[0].set_title('Original Image')
                axes[0].axis('off')

                im = axes[1].imshow(heatmap, cmap='jet')
                axes[1].set_title('Attention Map')
                axes[1].axis('off')
                cbar = plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
                cbar.set_ticks([0, 64, 128, 192, 255])
                cbar.set_ticklabels(['0.0', '0.2', '0.4', '0.6', '0.8'])

                axes[2].imshow(superimposed)
                axes[2].set_title('Grad-CAM Overlay')
                axes[2].axis('off')

                # Add prediction information
                true_class = tumor_types[label]
                pred_class = tumor_types[predicted_class]
                pred_prob = probs[0][predicted_class].item() * 100

                plt.suptitle(
                    f'True: {true_class}\nPredicted: {pred_class} ({pred_prob:.2f}%)',
                    y=1.05,
                    fontsize=12
                )

                plt.show()
                plt.close()

            except Exception as e:
                print(f"Error processing image at index {sample_idx}: {str(e)}")
                print(f"Activation shape: {grad_cam.activation.shape if grad_cam.activation is not None else 'None'}")
                print(f"Gradients shape: {grad_cam.gradients.shape if grad_cam.gradients is not None else 'None'}")
                traceback.print_exc()
                continue

In [ ]:
# Main execution code
if __name__ == "__main__":
    # 1. Setup transforms
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # 2. Create dataset
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)

    # 3. Load model
    model = initialize_levit_model(num_classes=4)
    model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/levit_384_model_final.pth')['model_state_dict'])
    model = model.cuda()
    model.eval()

    # 4. Visualize attention maps
    visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3)

# Model 5: CoAtNet-0

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
import timm
import time
import warnings
import pandas as pd
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score
import traceback

warnings.filterwarnings('ignore')
torch._dynamo.config.suppress_errors = True

class CoAtNet0Model(nn.Module):
    def __init__(self, num_classes):
        super(CoAtNet0Model, self).__init__()

        # Load pre-trained CoAtNet-0-RW model
        self.coatnet = timm.create_model('coatnet_0_rw_224', pretrained=True, num_classes=0)

        # Enable gradient checkpointing if available
        if hasattr(self.coatnet, 'set_grad_checkpointing'):
            self.coatnet.set_grad_checkpointing(enable=True)

        # Get the number of features from CoAtNet
        coatnet_num_features = self.coatnet.num_features

        # Add final classification layers with reduced size for memory efficiency
        self.fc = nn.Sequential(
            nn.Linear(coatnet_num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.coatnet(x)
        output = self.fc(features)
        return output

def initialize_coatnet_model(num_classes):
    return CoAtNet0Model(num_classes)

In [ ]:
def train_coatnet_model(train_dataset, test_dataset, num_classes, num_epochs=50, patience=5, k_folds=5):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    print("Training with pre-trained weights")

    # Set memory-efficient settings
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # Initialize gradient scaler for mixed precision
    scaler = GradScaler()

    criterion = nn.CrossEntropyLoss()

    # Slightly larger batch size since we're fine-tuning
    batch_size = 24

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=train_subsampler,
            num_workers=2,
            pin_memory=True
        )
        val_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=val_subsampler,
            num_workers=2,
            pin_memory=True
        )

        model = initialize_coatnet_model(num_classes).to(device)

        # Use different learning rates for pre-trained layers and new layers
        optimizer = optim.AdamW([
            {'params': model.coatnet.parameters(), 'lr': 1e-5},  # Lower learning rate for pre-trained layers
            {'params': model.fc.parameters(), 'lr': 1e-4}       # Higher learning rate for new layers
        ])

        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

        best_val_loss = float('inf')
        best_model = None
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            # Training phase
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    # Use mixed precision training
                    with autocast():
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    # Scale gradients and optimize
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                    train_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()

                    # Clear cache periodically
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()

                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        with autocast():
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            # Calculate average losses and accuracies
            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total

            print(f'Epoch {epoch+1}/{num_epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

            # Clear cache after each epoch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Save the final model
    torch.save({
        'epoch': len(results),
        'model_state_dict': best_model,
        'results': results
    }, 'final_pretrained_coatnet_0_model.pth')

    return results, model

In [ ]:
if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    try:
        # Create the datasets
        train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
        test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

        print("Dataset sizes:")
        print(f"Training: {len(train_dataset)}")
        print(f"Testing: {len(test_dataset)}")

        # Train the CoAtNet-0 model
        results, final_model = train_coatnet_model(train_dataset, test_dataset, num_classes)

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'coatnet_0_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Evaluate on the test set
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display metrics
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save metrics
        metrics_csv_path = os.path.join(base_path, 'coatnet_0_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save final model
        model_path = os.path.join(base_path, 'coatnet_0_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'coatnet_0_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()
        raise e

    print("\nCoAtNet-0 training, evaluation, and metrics logging complete!")

Since the comp complexity a.k.a time to train is too long I will make the educated guess to not experiment with more complex co-at-net models based on the fact that research shows that accuracy doesn't improve all that much.

# CoAtNet1

In [ ]:
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import timm
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
import time

warnings.filterwarnings('ignore')
torch._dynamo.config.suppress_errors = True

class CoAtNet1Model(nn.Module):
    def __init__(self, num_classes):
        super(CoAtNet1Model, self).__init__()

        # Load pre-trained CoAtNet-1-RW model
        self.coatnet = timm.create_model('coatnet_1_rw_224', pretrained=True, num_classes=0)

        # Enable gradient checkpointing if available
        if hasattr(self.coatnet, 'set_grad_checkpointing'):
            self.coatnet.set_grad_checkpointing(enable=True)

        # Get the number of features from CoAtNet-1
        coatnet_num_features = self.coatnet.num_features

        # Add final classification layers with reduced size for memory efficiency
        # Note: CoAtNet-1 has more features than CoAtNet-0, so we add an extra reduction layer
        self.fc = nn.Sequential(
            nn.Linear(coatnet_num_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.coatnet(x)
        output = self.fc(features)
        return output

def initialize_coatnet_model(num_classes):
    return CoAtNet1Model(num_classes)


In [ ]:
def train_coatnet_model(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    print("Training with pre-trained CoAtNet-1 weights")

    # Set memory-efficient settings
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # Initialize gradient scaler for mixed precision
    scaler = GradScaler()

    criterion = nn.CrossEntropyLoss()

    # Reduced batch size for CoAtNet-1 since it's larger
    batch_size = 16

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=train_subsampler,
            num_workers=2,
            pin_memory=True
        )
        val_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=val_subsampler,
            num_workers=2,
            pin_memory=True
        )

        model = initialize_coatnet_model(num_classes).to(device)

        # Adjusted learning rates for CoAtNet-1
        optimizer = optim.AdamW([
            {'params': model.coatnet.parameters(), 'lr': 5e-6},  # Lower learning rate for pre-trained layers
            {'params': model.fc.parameters(), 'lr': 5e-5}       # Lower learning rate for new layers
        ])

        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

        best_val_loss = float('inf')
        best_model = None
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            # Training phase
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    # Use mixed precision training
                    with autocast():
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    # Scale gradients and optimize
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                    train_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()

                    # Clear cache more frequently for CoAtNet-1
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()

                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        with autocast():
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            # Calculate average losses and accuracies
            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total

            print(f'Epoch {epoch+1}/{num_epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

            # Clear cache after each epoch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Save the final model
    torch.save({
        'epoch': len(results),
        'model_state_dict': best_model,
        'results': results
    }, 'final_pretrained_coatnet_1_model.pth')

    return results, model

In [ ]:
if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    print("Initializing CoAtNet-1 training pipeline...")
    print(f"Number of classes: {num_classes}")
    print(f"Tumor types: {tumor_types}")

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization for CoAtNet-1
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.RandomAdjustSharpness(0.2),  # Added for CoAtNet-1
            transforms.RandomAutocontrast(),        # Added for CoAtNet-1
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    try:
        # Create the datasets
        train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
        test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

        print("\nDataset sizes:")
        print(f"Training: {len(train_dataset)}")
        print(f"Testing: {len(test_dataset)}")

        # Train the CoAtNet-1 model
        print("\nInitiating CoAtNet-1 training...")
        results, final_model = train_coatnet_model(
            train_dataset,
            test_dataset,
            num_classes,
            num_epochs=50,    # Adjust these parameters as needed
            patience=5,
            k_folds=5
        )

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'coatnet_1_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"\nTraining history saved to {history_path}")

        # Evaluate on the test set
        print("\nEvaluating model on test set...")
        test_loader = DataLoader(
            test_dataset,
            batch_size=16,  # Reduced batch size for CoAtNet-1
            shuffle=False,
            num_workers=2,   # Reduced workers for memory efficiency
            pin_memory=True
        )

        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display metrics
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save metrics
        metrics_csv_path = os.path.join(base_path, 'coatnet_1_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save confusion matrix
        cm = confusion_matrix(true_labels, predictions)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=tumor_types,
                   yticklabels=tumor_types)
        plt.title('CoAtNet-1 Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(os.path.join(base_path, 'coatnet_1_confusion_matrix.png'))
        plt.close()

        # Save final model
        model_path = os.path.join(base_path, 'coatnet_1_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'coatnet_1_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

        # Print final summary
        print("\nFinal Summary:")
        print(f"Test Accuracy: {test_acc:.4f}")
        print(f"Average F1 Score: {np.mean(f1):.4f}")
        print(f"Average AUC-ROC: {np.mean(auc_roc):.4f}")
        print(f"Total Training Time: {results[-1]['Training Time (s)']:.2f} seconds")

    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        traceback.print_exc()
        raise e

    print("\nCoAtNet-1 training, evaluation, and metrics logging complete!")

# Model 9: XCiT_small_12

In [ ]:
import warnings
import torch
import torch.nn as nn
import timm
from torch.cuda.amp import autocast, GradScaler

warnings.filterwarnings('ignore')
torch._dynamo.config.suppress_errors = True

class XCiTModel(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(XCiTModel, self).__init__()

        # Load XCiT small_12 model with pretrained parameter
        self.xcit = timm.create_model('xcit_small_12_p8_224', pretrained=pretrained, num_classes=0)

        # Enable gradient checkpointing for memory efficiency
        self.xcit.set_grad_checkpointing(enable=True)

        # Get the number of features from XCiT
        xcit_num_features = self.xcit.num_features

        # Add final classification layers
        self.fc = nn.Sequential(
            nn.Linear(xcit_num_features, 512),  # Reduced from 1024 to 512 for memory efficiency
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.xcit(x)
        output = self.fc(features)
        return output

def initialize_xcit_model(num_classes, pretrained=True):
    return XCiTModel(num_classes, pretrained=pretrained)

In [ ]:
def train_xcit_model(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5, pretrained=True):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    print(f"Training with {'pretrained' if pretrained else 'randomly initialized'} weights")

    # Set memory-efficient settings
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # Initialize gradient scaler for mixed precision
    scaler = GradScaler()

    criterion = nn.CrossEntropyLoss()

    # Use smaller batch size to prevent OOM errors
    batch_size = 16  # Reduced from 32 but can be larger than XCiT-24 since model is smaller

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=train_subsampler,
            num_workers=2,
            pin_memory=True
        )
        val_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=val_subsampler,
            num_workers=2,
            pin_memory=True
        )

        model = initialize_xcit_model(num_classes, pretrained=pretrained).to(device)

        # Use different learning rates for pretrained vs non-pretrained
        initial_lr = 0.0001 if pretrained else 0.001
        optimizer = optim.Adam(model.parameters(), lr=initial_lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        best_val_loss = float('inf')
        best_model = None
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            # Training phase
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    # Use mixed precision training
                    with autocast():
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    # Scale gradients and optimize
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                    train_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()

                    # Clear cache periodically
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()

                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        with autocast():
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            # Calculate average losses and accuracies
            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total

            print(f'Epoch {epoch+1}/{num_epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            if best_model is None or val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

            # Clear cache after each epoch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Save the final model
    torch.save({
        'epoch': len(results),
        'model_state_dict': best_model,
        'results': results,
        'pretrained': pretrained
    }, 'final_xcit_small_12_model.pth')

    return results, model

In [ ]:
if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    try:
        # Create the datasets
        train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
        test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

        print("Dataset sizes:")
        print(f"Training: {len(train_dataset)}")
        print(f"Testing: {len(test_dataset)}")

        # Train the XCiT model
        results, final_model = train_xcit_model(train_dataset, test_dataset, num_classes)

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'xcit_small_12_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Evaluate on the test set
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display metrics
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save metrics
        metrics_csv_path = os.path.join(base_path, 'xcit_small_12_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save final model
        model_path = os.path.join(base_path, 'xcit_small_12_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'xcit_small_12_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()
        raise e

    print("\nXCiT Small 12 training, evaluation, and metrics logging complete!")

#LevitHybrid From timm Library:

Importing Contigencies:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from sklearn.model_selection import KFold
import time
import pandas as pd
import os
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import timm
from torchvision import models

In [ ]:
class LeViTEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes, levit_model='levit_384', pretrained=True):
        super(LeViTEfficientNetHybrid, self).__init__()

        # Load pretrained LeViT model
        self.levit = timm.create_model(levit_model, pretrained=pretrained, num_classes=0)

        # Load pretrained EfficientNet-B0
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        # Remove the final classification layers from EfficientNet
        self.efficientnet_features = nn.Sequential(*list(self.efficientnet.children())[:-1])

        # Get the number of features from both models
        levit_num_features = self.levit.num_features
        efficientnet_num_features = self.efficientnet.classifier[1].in_features

        # Combine features
        self.combined_features = levit_num_features + efficientnet_num_features

        # Add final classification layers
        self.fc = nn.Sequential(
            nn.Linear(self.combined_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Process input through LeViT
        levit_output = self.levit(x)

        # Process input through EfficientNet
        efficientnet_output = self.efficientnet_features(x)
        efficientnet_output = torch.flatten(efficientnet_output, 1)

        # Concatenate features
        combined = torch.cat((levit_output, efficientnet_output), dim=1)

        # Final classification
        output = self.fc(combined)
        return output

# Function to initialize the model
def initialize_levit_efficientnet_hybrid(num_classes, levit_model='levit_384', pretrained=True):
    return LeViTEfficientNetHybrid(num_classes, levit_model, pretrained)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
import time
import numpy as np

def train_levit_efficientnet_hybrid(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5, levit_model='levit_384'):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_subsampler, num_workers=2, pin_memory=True)
        val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_subsampler, num_workers=2, pin_memory=True)

        model = initialize_levit_efficientnet_hybrid(num_classes, levit_model).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        best_val_loss = float('inf')
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item() * inputs.size(0)
                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            val_acc = correct / total

            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                best_model = model.state_dict()
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Train on the entire dataset
    print('FINAL TRAINING')
    print('--------------------------------')
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    final_model = initialize_levit_efficientnet_hybrid(num_classes, levit_model).to(device)
    final_optimizer = optim.Adam(final_model.parameters(), lr=0.001)
    final_scheduler = ReduceLROnPlateau(final_optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    best_val_loss = float('inf')
    epochs_no_improve = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        final_model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                final_optimizer.zero_grad()
                outputs = final_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                final_optimizer.step()
                train_loss += loss.item() * inputs.size(0)
            except RuntimeError as e:
                print(f"RuntimeError in final training loop: {e}")
                print(f"Input shape: {inputs.shape}")
                continue

        final_model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = final_model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                except RuntimeError as e:
                    print(f"RuntimeError in final validation loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(test_loader.dataset)
        val_acc = correct / total

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        final_scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model = final_model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            break

    end_time = time.time()
    final_training_time = end_time - start_time
    results.append({
        'Fold': 'Final',
        'Best Validation Loss': best_val_loss,
        'Training Time (s)': final_training_time
    })

    # Save the final model
    torch.save(best_model, 'final_custom_levit_efficientnet_hybrid_model.pth')

    return results, final_model

In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader, tumor_types):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    all_preds = []
    all_labels = []
    all_probs = []
    total_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = (all_preds == all_labels).mean()
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    auc_roc = roc_auc_score(all_labels, all_probs, average='weighted', multi_class='ovr')
    avg_loss = total_loss / len(test_loader.dataset)

    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    print(f'AUC-ROC: {auc_roc:.4f}')
    print(f'Average Loss: {avg_loss:.4f}')

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=tumor_types, yticklabels=tumor_types)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return accuracy, precision, recall, f1, auc_roc, avg_loss, all_preds, all_labels

# Function to create metrics DataFrame
def create_metrics_dataframe(model, test_acc, precision, recall, f1, auc_roc, train_time, test_loss):
    metrics = {
        'Metric': ['Overall Accuracy', 'F1 Score', 'Cross Entropy Loss', 'Training Time (s)', 'Number of Parameters', 'Model Size (MB)'],
        'Value': [
            test_acc,
            f1,
            test_loss,
            train_time,
            sum(p.numel() for p in model.parameters()),
            sum(p.nelement() * p.element_size() for p in model.parameters()) / (1024 * 1024)
        ]
    }
    df = pd.DataFrame(metrics)
    return df



In [ ]:
if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # Create the datasets
    train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

    # Print dataset sizes
    print("Dataset sizes:")
    print(f"Training: {len(train_dataset)}")
    print(f"Testing: {len(test_dataset)}")

    try:
        # Train the Custom LeViT-ResNet model
        results, final_model = train_levit_efficientnet_hybrid(train_dataset, test_dataset, num_classes)

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'levit_efficientnet_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Evaluate on the test set
        test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display the metrics DataFrame
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save the metrics DataFrame
        metrics_csv_path = os.path.join(base_path, 'levit_efficientnet_hybrid_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save the final model with all relevant information
        model_path = os.path.join(base_path, 'levit_efficientnet_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'levit_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

    print("\nCustom LeViT-EffcientNet training, evaluation, and metrics logging complete!")

# LeVit compatible with GradCam

In [ ]:
class LeViTEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes, levit_model='levit_384', pretrained=True):
        super(LeViTEfficientNetHybrid, self).__init__()

        self.levit = timm.create_model(levit_model, pretrained=pretrained, num_classes=0)
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        self.efficientnet_features = nn.Sequential(*list(self.efficientnet.children())[:-1])

        levit_num_features = self.levit.num_features
        efficientnet_num_features = self.efficientnet.classifier[1].in_features
        self.combined_features = levit_num_features + efficientnet_num_features

        self.fc = nn.Sequential(
            nn.Linear(self.combined_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

        self.attention_maps = []
        self.attention_gradients = []

        def save_attention(name):
            def hook(module, input, output):
                self.attention_maps.append(output)
                # Register gradient hook for this output
                output.register_hook(lambda grad: self.attention_gradients.append(grad))
            return hook

        for name, module in self.levit.named_modules():
            if 'attn' in name:
                module.register_forward_hook(save_attention(name))

    def get_attention_maps(self):
        return self.attention_maps

    def get_attention_gradients(self):
        return self.attention_gradients

    def forward(self, x):
        self.attention_maps = []
        self.attention_gradients = []

        levit_output = self.levit(x)
        efficientnet_output = self.efficientnet_features(x)
        efficientnet_output = torch.flatten(efficientnet_output, 1)
        combined = torch.cat((levit_output, efficientnet_output), dim=1)
        return self.fc(combined)

def get_class_specific_attention(model, image, target_class, img_size=224):
    model.eval()

    # Forward pass with gradient
    image = image.clone().detach().requires_grad_(True)
    output = model(image)

    # Zero all gradients
    model.zero_grad()

    # Get the score for target class
    score = output[0, target_class]

    # Backward pass to get gradients
    score.backward()

    # Get attention maps and gradients
    attention_maps = model.get_attention_maps()
    attention_gradients = model.get_attention_gradients()

    # Get final stage attention (layer 89)
    final_attn = attention_maps[89]
    final_grad = attention_gradients[89] if len(attention_gradients) > 89 else torch.ones_like(final_attn)

    # Weight attention by gradients
    weighted_attn = (final_attn * final_grad).mean(dim=2)

    # Reshape to square
    attn_map = weighted_attn.reshape(4, 4)

    # Normalize
    attn_map = F.normalize(attn_map.view(-1), dim=0).view(4, 4)

    # Upsample to original image size
    attn_map = attn_map.unsqueeze(0).unsqueeze(0)
    attn_map = F.interpolate(attn_map, size=(img_size, img_size), mode='bilinear', align_corners=False)
    attn_map = attn_map.squeeze().detach().cpu().numpy()

    return attn_map

def initialize_levit_efficientnet_hybrid(num_classes, levit_model='levit_384', pretrained=True):
    return LeViTEfficientNetHybrid(num_classes, levit_model, pretrained)

In [ ]:
if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # Create the datasets
    train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

    # Print dataset sizes
    print("Dataset sizes:")
    print(f"Training: {len(train_dataset)}")
    print(f"Testing: {len(test_dataset)}")

    try:
        # Train the Custom LeViT-ResNet model
        results, final_model = train_levit_efficientnet_hybrid(train_dataset, test_dataset, num_classes)

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'levit_efficientnet_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Evaluate on the test set
        test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display the metrics DataFrame
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save the metrics DataFrame
        metrics_csv_path = os.path.join(base_path, 'levit_efficientnet_hybrid_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save the final model with all relevant information
        model_path = os.path.join(base_path, 'levit_efficientnet_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'levit_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

    print("\nCustom LeViT-EffcientNet training, evaluation, and metrics logging complete!")

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image

def visualize_attention(model, image_path, tumor_types):
    # Set model to eval mode
    model.eval()

    # Load and preprocess the image
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path)
    input_tensor = transform(image).unsqueeze(0)

    # Move to GPU if available
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)

    # Get prediction and attention maps
    with torch.no_grad():
        output = model(input_tensor)
        attention_maps = model.get_attention_maps()

    # Get predicted class
    pred = torch.argmax(output).item()
    pred_class = tumor_types[pred]

    # Process attention maps
    # We'll average attention across heads and create heatmaps
    attention_layers = []
    for attn in attention_maps:
        # Average across heads if necessary
        if len(attn.shape) > 3:
            attn = attn.mean(dim=1)
        attention_layers.append(attn.cpu().numpy())

    # Plotting
    n_layers = len(attention_layers)
    fig, axes = plt.subplots(2, (n_layers + 1) // 2, figsize=(15, 8))
    axes = axes.flatten()

    # Original image
    axes[0].imshow(image)
    axes[0].set_title('Original Image\nPredicted: ' + pred_class)
    axes[0].axis('off')

    # Attention maps
    for idx, attn in enumerate(attention_layers):
        # Reshape attention to square if necessary
        attn_map = attn[0]  # First batch
        h = int(np.sqrt(attn_map.shape[-1]))
        attn_map = attn_map.reshape(h, h)

        axes[idx+1].imshow(attn_map, cmap='viridis')
        axes[idx+1].set_title(f'Attention Layer {idx+1}')
        axes[idx+1].axis('off')

    plt.tight_layout()
    plt.show()

    return pred_class, attention_layers

# Example usage:
def analyze_sample_images(model, test_dataset, tumor_types, num_samples=5):
    """
    Analyze multiple sample images from the test dataset
    """
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    for i, (image, label) in enumerate(test_loader):
        if i >= num_samples:
            break

        true_class = tumor_types[label.item()]

        # Get prediction and attention maps
        with torch.no_grad():
            output = model(image.cuda())
            attention_maps = model.get_attention_maps()

        pred = torch.argmax(output).item()
        pred_class = tumor_types[pred]

        # Process attention maps
        attention_layers = []
        for attn in attention_maps:
            if len(attn.shape) > 3:
                attn = attn.mean(dim=1)
            attention_layers.append(attn.cpu().numpy())

        # Plotting
        n_layers = len(attention_layers)
        fig, axes = plt.subplots(2, (n_layers + 1) // 2, figsize=(15, 8))
        axes = axes.flatten()

        # Original image
        axes[0].imshow(image.squeeze().permute(1,2,0))
        axes[0].set_title(f'Original Image\nTrue: {true_class}\nPred: {pred_class}')
        axes[0].axis('off')

        # Attention maps
        for idx, attn in enumerate(attention_layers):
            attn_map = attn[0]
            h = int(np.sqrt(attn_map.shape[-1]))
            attn_map = attn_map.reshape(h, h)

            axes[idx+1].imshow(attn_map, cmap='viridis')
            axes[idx+1].set_title(f'Attention Layer {idx+1}')
            axes[idx+1].axis('off')

        plt.tight_layout()
        plt.show()

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

def analyze_sample_images_highres(model, test_dataset, tumor_types, num_samples=5, img_size=224):
    """
    Analyze samples with high resolution attention maps
    """
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    for i, (image, label) in enumerate(test_loader):
        if i >= num_samples:
            break

        if tumor_types[label.item()] == 'notumor':
            continue  # Skip no-tumor cases

        true_class = tumor_types[label.item()]

        # Get prediction and attention maps
        with torch.no_grad():
            output = model(image.cuda())
            attention_maps = model.get_attention_maps()

        pred = torch.argmax(output).item()
        pred_class = tumor_types[pred]

        # Select key layers from each stage
        key_layers = [0, 39, 89]  # First layer of each stage

        # Process attention maps
        processed_maps = []
        for layer_idx in key_layers:
            attn = attention_maps[layer_idx]

            # Remove batch dimension if present
            if len(attn.shape) == 3:
                attn = attn.squeeze(0)

            # Get number of tokens
            num_tokens = attn.shape[0]

            if num_tokens == 196:
                size = 14
            elif num_tokens == 49:
                size = 7
            elif num_tokens == 16:
                size = 4
            else:
                continue

            # Average attention patterns
            attn = attn.mean(dim=-1)  # Average over all attention heads
            attn = attn.reshape(size, size)

            # Upsample to original image size
            attn = torch.tensor(attn).unsqueeze(0).unsqueeze(0)
            attn = F.interpolate(attn, size=(img_size, img_size), mode='bilinear', align_corners=False)
            attn = attn.squeeze().cpu().numpy()

            processed_maps.append(attn)

        # Plotting
        fig, axes = plt.subplots(1, 4, figsize=(20, 5))

        # Original image
        img_display = image.squeeze().permute(1, 2, 0).cpu().numpy()
        # Denormalize
        img_display = img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img_display = np.clip(img_display, 0, 1)

        axes[0].imshow(img_display, cmap='gray')
        axes[0].set_title(f'Original Image\nTrue: {true_class}\nPred: {pred_class}')
        axes[0].axis('off')

        # Attention maps
        stage_names = ['Stage 1 Attention', 'Stage 2 Attention', 'Stage 3 Attention']
        for idx, (attn, stage_name) in enumerate(zip(processed_maps, stage_names)):
            axes[idx+1].imshow(img_display, cmap='gray', alpha=0.6)
            im = axes[idx+1].imshow(attn, cmap='jet', alpha=0.4)
            axes[idx+1].set_title(stage_name)
            axes[idx+1].axis('off')

        plt.tight_layout()
        plt.show()

        if true_class != pred_class:
            print("⚠️ Misclassified case!")
        print(f"True class: {true_class}")
        print(f"Predicted class: {pred_class}")
        print("-" * 50)

        # Only show one tumor case
        break

In [ ]:
def create_smooth_heatmap(attention_map, original_image, colormap=cv2.COLORMAP_INFERNO, alpha=0.4):
    """
    Create a smooth, visually appealing heatmap overlay

    Args:
        attention_map: The attention map from Grad-CAM
        original_image: Original input image
        colormap: OpenCV colormap to use (default: COLORMAP_INFERNO)
        alpha: Transparency of the heatmap overlay (default: 0.4)
    """
    # Apply Gaussian blur to smooth the attention map
    attention_map = cv2.GaussianBlur(attention_map, (3, 3), 0)

    # Resize attention map to match original image size
    heatmap = cv2.resize(attention_map, (original_image.shape[1], original_image.shape[0]))

    # Normalize to 0-255 range
    heatmap = np.uint8(255 * heatmap)

    # Apply colormap
    heatmap = cv2.applyColorMap(heatmap, colormap)

    # Convert to RGB
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Create superimposed image with smoother blending
    superimposed = cv2.addWeighted(original_image, 1 - alpha, heatmap, alpha, 0)

    # Enhance contrast of the superimposed image
    superimposed = cv2.convertScaleAbs(superimposed, alpha=1.2, beta=10)

    return heatmap, superimposed

def visualize_class_specific_attention(model, test_dataset, tumor_types, samples_per_class=3):
    """
    Visualize attention maps for specific samples from each class
    """
    model.eval()
    device = next(model.parameters()).device
    grad_cam = GradCAM(model)

    # Process samples from each class
    for class_idx in range(len(tumor_types)):
        print(f"\nProcessing class: {tumor_types[class_idx]}")

        # Get samples for this class
        class_samples = [idx for idx, (_, label) in enumerate(test_dataset) if label == class_idx]
        selected_samples = random.sample(class_samples, min(samples_per_class, len(class_samples)))

        for sample_idx in selected_samples:
            try:
                # Get image and create input tensor
                image, label = test_dataset[sample_idx]
                input_tensor = image.unsqueeze(0).to(device)

                # Generate attention map
                with torch.no_grad():
                    attention_map, output = grad_cam.generate_cam(input_tensor)
                    probs = F.softmax(output, dim=1)
                    predicted_class = output.argmax(dim=1).item()

                # Get original image
                original_image = cv2.imread(test_dataset.image_paths[sample_idx])
                original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

                # Create smooth heatmap visualization
                heatmap, superimposed = create_smooth_heatmap(
                    attention_map,
                    original_image,
                    colormap=cv2.COLORMAP_MAGMA,  # Using MAGMA colormap for better visualization
                    alpha=0.5
                )

                # Create figure with subplots
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))

                # Plot original image
                axes[0].imshow(original_image)
                axes[0].set_title('Original Image', fontsize=12)
                axes[0].axis('off')

                # Plot heatmap
                axes[1].imshow(heatmap)
                axes[1].set_title('Attention Map', fontsize=12)
                axes[1].axis('off')

                # Plot superimposed image
                axes[2].imshow(superimposed)
                axes[2].set_title('Superimposed', fontsize=12)
                axes[2].axis('off')

                # Add prediction information
                true_class = tumor_types[label]
                pred_class = tumor_types[predicted_class]
                pred_prob = probs[0][predicted_class].item() * 100

                plt.suptitle(
                    f'True: {true_class} | Predicted: {pred_class} ({pred_prob:.2f}%)\n'
                    f'Image: {test_dataset.image_paths[sample_idx].split("/")[-1]}',
                    fontsize=13,
                    y=1.02
                )

                # Adjust layout
                plt.tight_layout()

                # Add a white background
                fig.patch.set_facecolor('white')

                plt.show()
                plt.close()

            except Exception as e:
                print(f"Error processing image at index {sample_idx}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

In [ ]:
# 1. Setup transforms
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. Create dataset
test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)

# 3. Load model
model = initialize_levit_efficientnet_hybrid(num_classes=4, levit_model='levit_384')
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/levit_efficientnet_model_final.pth')['model_state_dict'])
model = model.cuda()

#4
visualize_class_specific_attention(model, test_dataset, tumor_types)

# CoAtNet with EffcientNetB0

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models

class CoAtNetEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes, coatnet_model='coatnet_0_rw_224.sw_in1k', pretrained=True):
        super(CoAtNetEfficientNetHybrid, self).__init__()

        # Load pretrained CoAtNet model
        self.coatnet = timm.create_model(coatnet_model, pretrained=pretrained, num_classes=0)

        # Load pretrained EfficientNet-B0
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        # Remove the final classification layers from EfficientNet
        self.efficientnet_features = nn.Sequential(*list(self.efficientnet.children())[:-1])

        # Get the number of features from both models
        coatnet_num_features = self.coatnet.num_features
        efficientnet_num_features = self.efficientnet.classifier[1].in_features

        # Combine features
        self.combined_features = coatnet_num_features + efficientnet_num_features

        # Add final classification layers
        self.fc = nn.Sequential(
            nn.Linear(self.combined_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Process input through CoAtNet
        coatnet_output = self.coatnet(x)

        # Process input through EfficientNet
        efficientnet_output = self.efficientnet_features(x)
        efficientnet_output = torch.flatten(efficientnet_output, 1)

        # Concatenate features
        combined = torch.cat((coatnet_output, efficientnet_output), dim=1)

        # Final classification
        output = self.fc(combined)
        return output

# Function to initialize the model
def initialize_coatnet_efficientnet_hybrid(num_classes, coatnet_model='coatnet_0_rw_224.sw_in1k', pretrained=True):
    return CoAtNetEfficientNetHybrid(num_classes, coatnet_model, pretrained)

In [ ]:
def train_coatnet_efficientnet_hybrid(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5, coatnet_model='coatnet_0_rw_224.sw_in1k'):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_subsampler, num_workers=2, pin_memory=True)
        val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_subsampler, num_workers=2, pin_memory=True)

        model = initialize_coatnet_efficientnet_hybrid(num_classes, coatnet_model).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        best_val_loss = float('inf')
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            for inputs, labels in train_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item() * inputs.size(0)
                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        continue

            train_loss = train_loss / len(train_loader.dataset)
            val_loss = val_loss / len(val_loader.dataset)
            val_acc = correct / total

            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                best_model = model.state_dict()
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print('Early stopping!')
                break

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

    # Train on the entire dataset
    print('FINAL TRAINING')
    print('--------------------------------')
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    final_model = initialize_coatnet_efficientnet_hybrid(num_classes, coatnet_model).to(device)
    final_optimizer = optim.Adam(final_model.parameters(), lr=0.001)
    final_scheduler = ReduceLROnPlateau(final_optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    best_val_loss = float('inf')
    epochs_no_improve = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        final_model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                final_optimizer.zero_grad()
                outputs = final_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                final_optimizer.step()
                train_loss += loss.item() * inputs.size(0)
            except RuntimeError as e:
                print(f"RuntimeError in final training loop: {e}")
                print(f"Input shape: {inputs.shape}")
                continue

        final_model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = final_model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                except RuntimeError as e:
                    print(f"RuntimeError in final validation loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    continue

        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(test_loader.dataset)
        val_acc = correct / total

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        final_scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model = final_model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            break

    end_time = time.time()
    final_training_time = end_time - start_time
    results.append({
        'Fold': 'Final',
        'Best Validation Loss': best_val_loss,
        'Training Time (s)': final_training_time
    })

    # Save the final model
    torch.save(best_model, 'final_custom_coatnet_efficientnet_hybrid_model.pth')

    return results, final_model

In [ ]:
if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Specify the CoAtNet model to use
    coatnet_model = 'coatnet_0_rw_224.sw_in1k'
    print(f"Using model: {coatnet_model}")

    # Data augmentation and normalization for training
    # Just normalization for validation/testing
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # Create the datasets
    train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

    # Print dataset sizes
    print("Dataset sizes:")
    print(f"Training: {len(train_dataset)}")
    print(f"Testing: {len(test_dataset)}")

    try:
        # Train the Custom CoAtNet-EfficientNet model
        results, final_model = train_coatnet_efficientnet_hybrid(
            train_dataset,
            test_dataset,
            num_classes,
            coatnet_model=coatnet_model
        )

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'coatnet_efficientnet_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Create test dataloader for final evaluation
        test_loader = DataLoader(
            test_dataset,
            batch_size=128,
            shuffle=False,
            num_workers=4,
            pin_memory=True
        )

        # Evaluate the model
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and display the metrics DataFrame
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string(index=False))

        # Save metrics
        metrics_csv_path = os.path.join(base_path, 'coatnet_efficientnet_hybrid_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save the final model with all relevant information
        model_path = os.path.join(base_path, 'coatnet_efficientnet_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'coatnet_model': coatnet_model,  # Save the specific CoAtNet model used
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'coatnet_test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

    print("\nCustom CoAtNet-EfficientNet training, evaluation, and metrics logging complete!")

# CoAtNet with GradCam

In [ ]:
# Function to initialize the model
def initialize_coatnet_efficientnet_hybrid(num_classes, coatnet_model='coatnet_0_rw_224.sw_in1k', pretrained=True):
    return CoAtNetEfficientNetHybrid(num_classes, coatnet_model, pretrained)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


class CoAtNetEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes, coatnet_model='coatnet_0_rw_224.sw_in1k', pretrained=True):
        super(CoAtNetEfficientNetHybrid, self).__init__()

        self.coatnet = timm.create_model(coatnet_model, pretrained=pretrained, num_classes=0)
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        self.efficientnet_features = nn.Sequential(*list(self.efficientnet.children())[:-1])

        coatnet_num_features = self.coatnet.num_features
        efficientnet_num_features = self.efficientnet.classifier[1].in_features
        self.combined_features = coatnet_num_features + efficientnet_num_features

        self.fc = nn.Sequential(
            nn.Linear(self.combined_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

        # Store activation and gradient
        self.last_conv_output = None
        self.last_conv_gradient = None

        # Register hooks
        def forward_hook(module, input, output):
            self.last_conv_output = output

        def backward_hook(module, grad_input, grad_output):
            self.last_conv_gradient = grad_output[0]

        # Find and register hooks for the last convolutional layer in CoAtNet
        for name, module in self.coatnet.named_modules():
            if isinstance(module, nn.Conv2d):
                module.register_forward_hook(forward_hook)
                module.register_full_backward_hook(backward_hook)

    def forward(self, x):
        coatnet_output = self.coatnet(x)
        efficientnet_output = self.efficientnet_features(x)
        efficientnet_output = torch.flatten(efficientnet_output, 1)
        combined = torch.cat((coatnet_output, efficientnet_output), dim=1)
        return self.fc(combined)

    def get_activation_gradient(self):
        return self.last_conv_gradient

    def get_activation(self):
        return self.last_conv_output



In [ ]:
import traceback

def modify_inplace_operations(model):
    """Disable inplace operations in the model"""
    for module in model.modules():
        if isinstance(module, nn.ReLU):
            module.inplace = False
        elif hasattr(module, 'act') and isinstance(module.act, nn.ReLU):
            module.act.inplace = False


class ModifiedCoAtNetEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes, coatnet_model='coatnet_0_rw_224.sw_in1k', pretrained=True):
        super().__init__()

        self.coatnet = timm.create_model(coatnet_model, pretrained=pretrained, num_classes=0)
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        self.efficientnet_features = nn.Sequential(*list(self.efficientnet.children())[:-1])

        coatnet_num_features = self.coatnet.num_features
        efficientnet_num_features = self.efficientnet.classifier[1].in_features
        self.combined_features = coatnet_num_features + efficientnet_num_features

        self.fc = nn.Sequential(
            nn.Linear(self.combined_features, 512),
            nn.ReLU(inplace=False),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

        # Register hooks for feature extraction
        def hook_fn(module, input, output):
            module.latest_features = output.clone()

        def grad_hook_fn(module, grad_input, grad_output):
            module.latest_gradients = grad_output[0].clone()

        for module in self.coatnet.modules():
            if isinstance(module, nn.Conv2d):
                module.register_forward_hook(hook_fn)
                module.register_full_backward_hook(grad_hook_fn)

    def forward(self, x):
        x = x.clone()  # Clone input to avoid inplace modifications
        coatnet_output = self.coatnet(x)
        efficientnet_output = self.efficientnet_features(x)
        efficientnet_output = torch.flatten(efficientnet_output, 1)
        combined = torch.cat((coatnet_output, efficientnet_output), dim=1)
        return self.fc(combined)

In [ ]:
class FeatureExtractor:
    """Helper class to extract features and gradients"""
    def __init__(self):
        self.features = None
        self.gradients = None

    def save_features(self, module, input, output):
        self.features = output.detach()

    def save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

def get_last_conv_layer(model):
    """Get the last convolutional layer in the model"""
    last_conv_layer = None
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            last_conv_layer = module
    return last_conv_layer

def visualize_test_folder(test_folder, model_path, num_samples=5):
    """Wrapper function for visualization"""
    # Setup transforms
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create dataset
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)

    # Initialize the base model
    model = initialize_coatnet_efficientnet_hybrid(num_classes=len(tumor_types))

    # Load state dict
    try:
        checkpoint = torch.load(model_path, map_location='cuda', weights_only=True)
    except:
        print("Warning: Failed with weights_only=True, trying regular load...")
        checkpoint = torch.load(model_path, map_location='cuda')

    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.cuda()
    model.eval()

    # Turn off all inplace operations
    for module in model.modules():
        if isinstance(module, nn.ReLU):
            module.inplace = False

    visualize_hybrid_attention(model, test_dataset, tumor_types, num_samples)

In [ ]:
def visualize_hybrid_attention(model, test_dataset, tumor_types, num_samples=5):
    """Visualize attention maps"""
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    for i, (image, label) in enumerate(test_loader):
        if i >= num_samples:
            break

        try:
            # Move to CUDA and create a copy for attention computation
            image_display = image.clone()  # For display
            image = image.cuda()  # For computation
            true_class = tumor_types[label.item()]

            # Get prediction
            with torch.no_grad():
                output = model(image)
            pred = torch.argmax(output).item()
            pred_class = tumor_types[pred]

            # Get attention maps
            coatnet_attn = get_coatnet_attention(model, image, label.item())
            cnn_gradcam = get_gradcam(model, image, label.item())

            # Create visualization
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Original image - use image_display instead of image
            img_display = image_display.squeeze().permute(1, 2, 0).detach().numpy()
            img_display = img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            img_display = np.clip(img_display, 0, 1)

            # Plot original
            axes[0].imshow(img_display)
            axes[0].set_title(f'Original Image\nTrue: {true_class}\nPred: {pred_class}')
            axes[0].axis('off')

            # Plot CoAtNet attention
            axes[1].imshow(img_display, alpha=0.6)
            im1 = axes[1].imshow(coatnet_attn, cmap='jet', alpha=0.4)
            axes[1].set_title('CoAtNet Attention')
            axes[1].axis('off')
            plt.colorbar(im1, ax=axes[1])

            # Plot EfficientNet GradCAM
            axes[2].imshow(img_display, alpha=0.6)
            im2 = axes[2].imshow(cnn_gradcam, cmap='jet', alpha=0.4)
            axes[2].set_title('EfficientNet GradCAM')
            axes[2].axis('off')
            plt.colorbar(im2, ax=axes[2])

            plt.tight_layout()
            plt.show()

            if true_class != pred_class:
                print("⚠️ Misclassified case!")
            print(f"True class: {true_class}")
            print(f"Predicted class: {pred_class}")
            print("-" * 50)

        except Exception as e:
            print(f"Error processing image {i}: {str(e)}")
            traceback.print_exc()
            continue

def get_coatnet_attention(model, input_image, class_idx):
    """Generate attention map for CoAtNet"""
    model.eval()

    # Create feature extractor
    feature_extractor = FeatureExtractor()

    # Get last conv layer
    last_conv = get_last_conv_layer(model.coatnet)
    if last_conv is None:
        raise ValueError("Could not find convolutional layer in model")

    # Register hooks
    forward_handle = last_conv.register_forward_hook(feature_extractor.save_features)
    backward_handle = last_conv.register_full_backward_hook(feature_extractor.save_gradients)

    try:
        # Forward pass
        input_tensor = input_image.clone().detach().requires_grad_(True)
        output = model(input_tensor)

        if output.dim() == 1:
            output = output.unsqueeze(0)

        # One-hot encoding for the target class
        one_hot = torch.zeros_like(output)
        one_hot[0, class_idx] = 1

        # Backward pass
        model.zero_grad()
        output.backward(gradient=one_hot)

        # Get feature maps and gradients
        features = feature_extractor.features
        gradients = feature_extractor.gradients

        if features is None or gradients is None:
            raise ValueError("Features or gradients were not captured")

        # Calculate attention weights
        pooled_gradients = torch.mean(gradients, dim=[2, 3], keepdim=True)
        attention = features * pooled_gradients
        attention = torch.sum(attention, dim=1, keepdim=True)

        # Apply ReLU and normalize
        attention = F.relu(attention)
        attention = F.interpolate(attention,
                                size=(224, 224),
                                mode='bilinear',
                                align_corners=False)

        attention = attention.squeeze().cpu().detach().numpy()
        attention = (attention - np.min(attention)) / (np.max(attention) - np.min(attention) + 1e-8)

        return attention

    finally:
        # Remove hooks
        forward_handle.remove()
        backward_handle.remove()

In [ ]:
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
model_path = '/content/drive/MyDrive/Colab Notebooks/coatnet_efficientnet_model_final.pth'

# Run visualization
visualize_test_folder(test_folder, model_path, num_samples=5)

In [ ]:
def get_fc_gradcam(model, image, target_class):
    """Get GradCAM-like visualization for the final fully connected layers"""
    model.eval()

    # Store activations and gradients
    activations = {}
    gradients = {}

    def save_activation(name):
        def hook(module, input, output):
            activations[name] = output
        return hook

    def save_gradient(name):
        def hook(module, grad_in, grad_out):
            gradients[name] = grad_out[0]
        return hook

    # Register hooks for both CNN and Transformer features before they're combined
    model.coatnet.register_forward_hook(save_activation('coatnet'))
    model.efficientnet_features.register_forward_hook(save_activation('efficientnet'))

    # Forward pass
    image.requires_grad_()
    output = model(image)
    score = output[0, target_class]

    # Backward pass
    model.zero_grad()
    score.backward()

    # Get the combined importance from both branches
    coatnet_importance = None
    if 'coatnet' in activations and 'coatnet' in gradients:
        coatnet_act = activations['coatnet']
        coatnet_grad = gradients['coatnet']
        coatnet_importance = (coatnet_act * coatnet_grad).sum(dim=1, keepdim=True)

    efficientnet_importance = None
    if 'efficientnet' in activations and 'efficientnet' in gradients:
        eff_act = activations['efficientnet']
        eff_grad = gradients['efficientnet']
        efficientnet_importance = (eff_act * eff_grad).sum(dim=1, keepdim=True)

    # Combine importances
    if coatnet_importance is not None and efficientnet_importance is not None:
        combined_importance = coatnet_importance + efficientnet_importance
    else:
        combined_importance = coatnet_importance if coatnet_importance is not None else efficientnet_importance

    # Apply ReLU to focus on features that positively influence the target class
    combined_importance = F.relu(combined_importance)

    # Interpolate to original image size
    combined_importance = F.interpolate(
        combined_importance,
        size=(224, 224),
        mode='bilinear',
        align_corners=False
    )

    # Normalize
    combined_importance = combined_importance.squeeze().detach().cpu().numpy()
    combined_importance = (combined_importance - np.min(combined_importance)) / \
                         (np.max(combined_importance) - np.min(combined_importance) + 1e-8)

    return combined_importance

def visualize_all_cams(model, image, label, tumor_types):
    """Visualize original GradCAM, FC GradCAM, and combined view"""
    # Get predictions
    with torch.no_grad():
        output = model(image)
    pred = torch.argmax(output).item()

    # Get different attention maps
    gradcam = get_gradcam(model, image, label.item())
    fc_gradcam = get_fc_gradcam(model, image, label.item())

    # Plotting
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))

    # Original image
    img_display = image.cpu().squeeze().permute(1, 2, 0).detach().numpy()
    img_display = img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img_display = np.clip(img_display, 0, 1)

    # Plot original
    axes[0].imshow(img_display)
    axes[0].set_title(f'Original Image\nTrue: {tumor_types[label.item()]}\nPred: {tumor_types[pred]}')
    axes[0].axis('off')

    # Plot CNN GradCAM
    axes[1].imshow(img_display, alpha=0.6)
    im1 = axes[1].imshow(gradcam, cmap='jet', alpha=0.4)
    axes[1].set_title('CNN GradCAM')
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1])

    # Plot FC GradCAM
    axes[2].imshow(img_display, alpha=0.6)
    im2 = axes[2].imshow(fc_gradcam, cmap='jet', alpha=0.4)
    axes[2].set_title('FC GradCAM')
    axes[2].axis('off')
    plt.colorbar(im2, ax=axes[2])

    # Plot Combined (average of both)
    combined = (gradcam + fc_gradcam) / 2
    axes[3].imshow(img_display, alpha=0.6)
    im3 = axes[3].imshow(combined, cmap='jet', alpha=0.4)
    axes[3].set_title('Combined View')
    axes[3].axis('off')
    plt.colorbar(im3, ax=axes[3])

    plt.tight_layout()
    plt.show()

def visualize_test_folder_all_cams(test_folder, model_path, num_samples=5):
    """Wrapper function to visualize all attention maps from test folder"""
    # Setup transforms
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create dataset
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)

    # Initialize model
    model = initialize_coatnet_efficientnet_hybrid(num_classes=len(tumor_types))

    # Load state dict
    checkpoint = torch.load(model_path, map_location='cuda')
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.cuda()
    model.eval()

    # Create data loader
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    # Visualize samples
    for i, (image, label) in enumerate(test_loader):
        if i >= num_samples:
            break

        try:
            image = image.cuda()
            visualize_all_cams(model, image, label, tumor_types)

        except Exception as e:
            print(f"Error processing image {i}: {str(e)}")
            traceback.print_exc()
            continue


In [ ]:
def disable_inplace_operations(model):
    """Disable all inplace operations in the model"""
    for module in model.modules():
        if hasattr(module, 'inplace'):
            module.inplace = False
        # Handle squeeze-excite layers specifically
        if hasattr(module, 'act') and hasattr(module.act, 'inplace'):
            module.act.inplace = False


In [ ]:
def get_fc_gradcam(model, input_image, target_class):
    """Generate attention map focusing on the last convolutional layer before FC"""
    model.eval()

    # Store the activations and gradients
    activations = {}
    gradients = {}

    def save_activation(module, input, output):
        activations['features'] = output.detach()

    def save_gradient(module, grad_input, grad_output):
        gradients['features'] = grad_output[0].detach()

    # Find the last convolutional layer before the FC layers
    last_conv = None
    for module in model.efficientnet.modules():
        if isinstance(module, nn.Conv2d):
            last_conv = module

    if last_conv is None:
        raise ValueError("Could not find convolutional layer")

    # Register hooks
    handle1 = last_conv.register_forward_hook(save_activation)
    handle2 = last_conv.register_full_backward_hook(save_gradient)

    try:
        # Forward pass
        input_tensor = input_image.clone().requires_grad_()
        output = model(input_tensor)

        if not activations['features'].requires_grad:
            activations['features'].requires_grad = True

        # Target for backprop
        one_hot = torch.zeros_like(output)
        one_hot[0, target_class] = 1

        # Zero gradients
        model.zero_grad()

        # Backward pass
        output.backward(gradient=one_hot, retain_graph=True)

        # Get weights based on global average pooling of gradients
        weights = torch.mean(gradients['features'], dim=(2, 3), keepdim=True)

        # Get weighted combination of forward activation maps
        cam = torch.mul(activations['features'], weights).sum(dim=1, keepdim=True)

        # ReLU
        cam = F.relu(cam)

        # Interpolate to image size
        cam = F.interpolate(cam, size=(224, 224), mode='bilinear', align_corners=False)

        # Convert to numpy and normalize - with proper detachment
        cam = cam.squeeze().detach().cpu().numpy()
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam) + 1e-8)

        return cam

    finally:
        handle1.remove()
        handle2.remove()

def visualize_test_folder_all_cams(test_folder, model_path, num_samples=5):
    """Wrapper function to visualize all attention maps from test folder"""
    # Setup transforms
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create dataset and loader
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    # Initialize model
    model = initialize_coatnet_efficientnet_hybrid(num_classes=len(tumor_types))

    # Disable inplace operations
    disable_inplace_operations(model)

    # Load state dict
    try:
        checkpoint = torch.load(model_path, map_location='cuda', weights_only=True)
    except:
        print("Warning: Failed with weights_only=True, trying regular load...")
        checkpoint = torch.load(model_path, map_location='cuda')

    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.cuda()
    model.eval()

    for i, (image, label) in enumerate(test_loader):
        if i >= num_samples:
            break

        try:
            image = image.cuda()
            true_class = tumor_types[label.item()]

            # Get prediction
            with torch.no_grad():
                output = model(image)
            pred = torch.argmax(output).item()
            pred_class = tumor_types[pred]

            # Get attention maps
            gradcam = get_gradcam(model, image.clone(), label.item())
            fc_gradcam = get_fc_gradcam(model, image.clone(), label.item())

            # Plotting
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Original image - with proper detachment
            img_display = image.cpu().squeeze().permute(1, 2, 0).detach().numpy()
            img_display = img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            img_display = np.clip(img_display, 0, 1)

            # Plot original
            axes[0].imshow(img_display)
            axes[0].set_title(f'Original Image\nTrue: {true_class}\nPred: {pred_class}')
            axes[0].axis('off')

            # Plot CNN GradCAM
            axes[1].imshow(img_display, alpha=0.6)
            im1 = axes[1].imshow(gradcam, cmap='jet', alpha=0.4)
            axes[1].set_title('CNN GradCAM')
            axes[1].axis('off')
            plt.colorbar(im1, ax=axes[1])

            # Plot FC GradCAM
            axes[2].imshow(img_display, alpha=0.6)
            im2 = axes[2].imshow(fc_gradcam, cmap='jet', alpha=0.4)
            axes[2].set_title('FC GradCAM')
            axes[2].axis('off')
            plt.colorbar(im2, ax=axes[2])

            plt.tight_layout()
            plt.show()

            if true_class != pred_class:
                print("⚠️ Misclassified case!")
            print(f"True class: {true_class}")
            print(f"Predicted class: {pred_class}")
            print("-" * 50)

        except Exception as e:
            print(f"Error processing image {i}: {str(e)}")
            traceback.print_exc()
            continue

In [ ]:
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
model_path = '/content/drive/MyDrive/Colab Notebooks/coatnet_efficientnet_model_final.pth'
visualize_test_folder_all_cams(test_folder, model_path, num_samples=5)

In [ ]:
def visualize_test_folder_all_cams(test_folder, model_path, num_samples=5):
    """Enhanced visualization with multiple color schemes"""
    # Setup transforms
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create dataset and loader
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    # Initialize model
    model = initialize_coatnet_efficientnet_hybrid(num_classes=len(tumor_types))
    disable_inplace_operations(model)

    # Load state dict
    checkpoint = torch.load(model_path, map_location='cuda')
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.cuda()
    model.eval()

    # Different colormaps to try
    colormaps = ['jet', 'hot', 'plasma']

    for i, (image, label) in enumerate(test_loader):
        if i >= num_samples:
            break

        try:
            image = image.cuda()
            true_class = tumor_types[label.item()]

            # Get prediction
            with torch.no_grad():
                output = model(image)
            pred = torch.argmax(output).item()
            pred_class = tumor_types[pred]

            # Get attention maps
            gradcam = get_gradcam(model, image.clone(), label.item())
            fc_gradcam = get_fc_gradcam(model, image.clone(), label.item())

            # Create multiple visualizations with different parameters
            fig, axes = plt.subplots(2, 3, figsize=(18, 10))

            # Original image
            img_display = image.cpu().squeeze().permute(1, 2, 0).detach().numpy()
            img_display = img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            img_display = np.clip(img_display, 0, 1)

            # Row 1: Original colormaps
            axes[0, 0].imshow(img_display, cmap='gray')
            axes[0, 0].set_title(f'Original Image\nTrue: {true_class}\nPred: {pred_class}')
            axes[0, 0].axis('off')

            # CNN GradCAM with jet colormap
            axes[0, 1].imshow(img_display, cmap='gray', alpha=0.7)
            im1 = axes[0, 1].imshow(gradcam, cmap='jet', alpha=0.5)
            axes[0, 1].set_title('CNN GradCAM (jet)')
            axes[0, 1].axis('off')
            plt.colorbar(im1, ax=axes[0, 1])

            # FC GradCAM with jet colormap
            axes[0, 2].imshow(img_display, cmap='gray', alpha=0.7)
            im2 = axes[0, 2].imshow(fc_gradcam, cmap='jet', alpha=0.5)
            axes[0, 2].set_title('FC GradCAM (jet)')
            axes[0, 2].axis('off')
            plt.colorbar(im2, ax=axes[0, 2])

            # Row 2: Alternative visualizations
            # Heatmap only - CNN GradCAM
            axes[1, 0].imshow(gradcam, cmap='hot')
            axes[1, 0].set_title('CNN GradCAM Heatmap Only (hot)')
            axes[1, 0].axis('off')

            # Heatmap only - FC GradCAM
            axes[1, 1].imshow(fc_gradcam, cmap='plasma')
            axes[1, 1].set_title('FC GradCAM Heatmap Only (plasma)')
            axes[1, 1].axis('off')

            # Combined visualization
            combined = (gradcam + fc_gradcam) / 2
            axes[1, 2].imshow(img_display, cmap='gray', alpha=0.7)
            im3 = axes[1, 2].imshow(combined, cmap='viridis', alpha=0.5)
            axes[1, 2].set_title('Combined Attention')
            axes[1, 2].axis('off')
            plt.colorbar(im3, ax=axes[1, 2])

            plt.tight_layout()
            plt.show()

            # Print additional information
            print(f"True class: {true_class}")
            print(f"Predicted class: {pred_class}")
            print(f"Max CNN attention value: {np.max(gradcam):.3f}")
            print(f"Max FC attention value: {np.max(fc_gradcam):.3f}")
            print("-" * 50)

        except Exception as e:
            print(f"Error processing image {i}: {str(e)}")
            traceback.print_exc()
            continue

In [ ]:
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
model_path = '/content/drive/MyDrive/Colab Notebooks/coatnet_efficientnet_model_final.pth'
visualize_test_folder_all_cams(test_folder, model_path, num_samples=5)

#Cross-Covariance Image Transformers w/ EfficientNetB0

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models
import gc
from torch.amp import autocast, GradScaler

class XCiTEfficientNetHybrid(nn.Module):
    def __init__(self, num_classes, xcit_model='xcit_small_24_p8_224', pretrained=True):
        super(XCiTEfficientNetHybrid, self).__init__()

        # Load pretrained XCiT model
        self.xcit = timm.create_model(xcit_model, pretrained=pretrained, num_classes=0)

        # Load pretrained EfficientNet-B0
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        # Remove the final classification layers from EfficientNet
        self.efficientnet_features = nn.Sequential(*list(self.efficientnet.children())[:-1])

        # Get the number of features from both models
        xcit_num_features = self.xcit.num_features
        efficientnet_num_features = self.efficientnet.classifier[1].in_features

        # Combine features
        self.combined_features = xcit_num_features + efficientnet_num_features

        # Add final classification layers with modified batch normalization
        self.fc = nn.Sequential(
            nn.Linear(self.combined_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Process input through XCiT
        xcit_output = self.xcit(x)

        # Process input through EfficientNet
        efficientnet_output = self.efficientnet_features(x)
        efficientnet_output = torch.flatten(efficientnet_output, 1)

        # Concatenate features
        combined = torch.cat((xcit_output, efficientnet_output), dim=1)

        # Final classification
        output = self.fc(combined)
        return output

def initialize_xcit_efficientnet_hybrid(num_classes, xcit_model='xcit_small_24_p8_224', pretrained=True):
    return XCiTEfficientNetHybrid(num_classes, xcit_model, pretrained)


In [ ]:
def train_xcit_efficientnet_hybrid(train_dataset, test_dataset, num_classes, num_epochs=100, patience=10, k_folds=5, xcit_model='xcit_small_24_p8_224'):
    results = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Memory optimization settings
    torch.cuda.empty_cache()
    gc.collect()

    # Correct GradScaler initialization
    scaler = torch.amp.GradScaler()

    # Adjusted batch sizes and accumulation
    batch_size = 16
    accumulation_steps = 2

    # K-Fold Cross-validation
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        train_subsampler = SubsetRandomSampler(train_ids)
        val_subsampler = SubsetRandomSampler(val_ids)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=train_subsampler,
            num_workers=2,
            pin_memory=True,
            drop_last=True
        )

        val_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=val_subsampler,
            num_workers=2,
            pin_memory=True,
            drop_last=True
        )

        model = initialize_xcit_efficientnet_hybrid(num_classes, xcit_model).to(device)

        # Fixed gradient checkpointing call
        if hasattr(model.xcit, 'set_grad_checkpointing'):
            model.xcit.set_grad_checkpointing(True)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        best_val_loss = float('inf')
        epochs_no_improve = 0
        start_time = time.time()

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            optimizer.zero_grad()

            for i, (inputs, labels) in enumerate(train_loader):
                try:
                    inputs, labels = inputs.to(device), labels.to(device)

                    # Updated autocast usage
                    with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss = loss / accumulation_steps

                    scaler.scale(loss).backward()

                    if (i + 1) % accumulation_steps == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        optimizer.zero_grad()

                    train_loss += loss.item() * inputs.size(0) * accumulation_steps

                    del outputs, loss
                    torch.cuda.empty_cache()

                except RuntimeError as e:
                    print(f"RuntimeError in training loop: {e}")
                    print(f"Input shape: {inputs.shape}")
                    torch.cuda.empty_cache()
                    continue

            # Validation phase
            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    try:
                        inputs, labels = inputs.to(device), labels.to(device)

                        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)

                        val_loss += loss.item() * inputs.size(0)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                        del outputs, loss
                        torch.cuda.empty_cache()

                    except RuntimeError as e:
                        print(f"RuntimeError in validation loop: {e}")
                        print(f"Input shape: {inputs.shape}")
                        torch.cuda.empty_cache()
                        continue

            if total > 0:
                train_loss = train_loss / len(train_loader.dataset)
                val_loss = val_loss / total
                val_acc = correct / total

                print(f'Epoch {epoch+1}/{num_epochs}')
                print(f'Train Loss: {train_loss:.4f}')
                print(f'Val Loss: {val_loss:.4f}')
                print(f'Val Acc: {val_acc:.4f}')

                scheduler.step(val_loss)

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    epochs_no_improve = 0
                    best_model = model.state_dict()
                else:
                    epochs_no_improve += 1

                if epochs_no_improve == patience:
                    print('Early stopping!')
                    break

        end_time = time.time()
        training_time = end_time - start_time
        results.append({
            'Fold': fold+1,
            'Best Validation Loss': best_val_loss,
            'Training Time (s)': training_time
        })

        # Clean up memory after each fold
        del model
        torch.cuda.empty_cache()
        gc.collect()

    # Load the final model for return
    final_model = initialize_xcit_efficientnet_hybrid(num_classes, xcit_model).to(device)
    final_model.load_state_dict(best_model)

    return results, final_model

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix

if __name__ == "__main__":
    # Setup
    base_path = '/content/drive/MyDrive/Colab Notebooks'
    train_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Training'
    test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
    tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']
    num_classes = len(tumor_types)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(base_path, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Specify the XCiT model to use
    xcit_model = 'xcit_small_24_p8_224'
    print(f"Using model: {xcit_model}")

    # Data augmentation and normalization for training
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # Create the datasets
    train_dataset = MRIDataset(train_folder, tumor_types, transform=data_transforms['train'])
    test_dataset = MRIDataset(test_folder, tumor_types, transform=data_transforms['val'])

    # Print dataset sizes
    print("Dataset sizes:")
    print(f"Training: {len(train_dataset)}")
    print(f"Testing: {len(test_dataset)}")

    try:
        # Train the model
        results, final_model = train_xcit_efficientnet_hybrid(
            train_dataset,
            test_dataset,
            num_classes,
            xcit_model=xcit_model
        )

        # Display training results
        results_df = pd.DataFrame(results)
        print("\nTraining Results:")
        print(results_df)

        # Save training history
        history_path = os.path.join(base_path, 'xcit_efficientnet_training_history.csv')
        results_df.to_csv(history_path, index=False)
        print(f"Training history saved to {history_path}")

        # Create test dataloader for final evaluation
        test_loader = DataLoader(
            test_dataset,
            batch_size=128,
            shuffle=False,
            num_workers=4,
            pin_memory=True
        )

        # Evaluate the model
        test_acc, precision, recall, f1, auc_roc, test_loss, predictions, true_labels = evaluate_model(
            final_model,
            test_loader,
            tumor_types
        )

        # Create and save metrics DataFrame
        metrics_df = create_metrics_dataframe(
            final_model,
            test_acc,
            precision,
            recall,
            f1,
            auc_roc,
            results[-1]['Training Time (s)'],
            test_loss
        )
        print("\nModel Metrics:")
        print(metrics_df.to_string())

        # Save metrics
        metrics_csv_path = os.path.join(base_path, 'xcit_efficientnet_model_metrics.csv')
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"\nMetrics saved to {metrics_csv_path}")

        # Save the final model
        model_path = os.path.join(base_path, 'xcit_efficientnet_model_final.pth')
        torch.save({
            'epoch': len(results),
            'model_state_dict': final_model.state_dict(),
            'results': results,
            'test_metrics': {
                'accuracy': test_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'auc_roc': auc_roc,
                'test_loss': test_loss
            }
        }, model_path)
        print(f"\nModel saved to {model_path}")

        # Save predictions
        predictions_df = pd.DataFrame({
            'True_Label': [tumor_types[i] for i in true_labels],
            'Predicted_Label': [tumor_types[i] for i in predictions]
        })
        predictions_path = os.path.join(base_path, 'test_predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)
        print(f"Test predictions saved to {predictions_path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

    print("\nXCiT-EfficientNet training, evaluation, and metrics logging complete!")

# Attention Visualization:


### Loading the models we trained :



In [ ]:
# Efficient Net B0

cnn _model = initialize_model(num_classes=len(tumor_types))
cnn_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/final_efficientnet_b0_mri_classification_model.pth'))
cnn_model.eval()

In [ ]:
# LeVit

levit_model = initialize_levit_efficientnet_hybrid(num_classes=len(tumor_types))
levit_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/levit_efficientnet_model_final.pth')['model_state_dict'])
levit_model.eval()

In [ ]:
#CoAtNet

coatnet_model = initialize_coatnet_efficientnet_hybrid(num_classes=len(tumor_types))
coatnet_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/coatnet_efficientnet_model_final.pth')['model_state_dict'])
coatnet_model.eval()

In [ ]:
#XCIT

xcit_model = initialize_xcit_efficientnet_hybrid(num_classes=len(tumor_types))
xcit_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/xcit_efficientnet_model_final.pth')['model_state_dict'])
xcit_model.eval()

## Grad-Cam:

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

class BaseGradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.gradients = None
        self.activations = None
        self.target_layer = target_layer

        def forward_hook(module, input, output):
            self.activations = output.detach()

        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()

        # Using register_full_backward_hook instead of register_backward_hook
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_full_backward_hook(backward_hook)

    def get_cam(self, input_tensor):
        # Forward pass
        self.model.zero_grad()
        output = self.model(input_tensor)
        pred = output.argmax(dim=1).item()

        # Get class activation
        one_hot = torch.zeros_like(output)
        one_hot[0][pred] = 1

        # Backward pass
        output.backward(gradient=one_hot, retain_graph=True)

        return pred, output.softmax(dim=1)[0].detach().cpu().numpy()

class EfficientNetGradCAM(BaseGradCAM):
    def __init__(self, model):
        super().__init__(model, model.features[7][0].block[1])

    def __call__(self, input_tensor):
        pred, class_probs = self.get_cam(input_tensor)

        weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = F.interpolate(cam, size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

        return cam.squeeze().cpu().numpy(), pred, class_probs

class LeViTGradCAM(BaseGradCAM):
    def __init__(self, model):
        super().__init__(model, model.levit.stages[-1].blocks[-1].attn)

    def __call__(self, input_tensor):
        pred, class_probs = self.get_cam(input_tensor)

        # Handle attention maps
        b, n, c = self.activations.shape
        h = w = int(np.sqrt(n - 1))  # Subtract 1 for CLS token

        # Remove CLS token and reshape
        activations = self.activations[:, 1:, :].reshape(b, h, w, c).permute(0, 3, 1, 2)
        gradients = self.gradients[:, 1:, :].reshape(b, h, w, c).permute(0, 3, 1, 2)

        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
        cam = torch.sum(weights * activations, dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = F.interpolate(cam, size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

        return cam.squeeze().cpu().numpy(), pred, class_probs

class CoAtNetGradCAM(BaseGradCAM):
    def __init__(self, model):
        super().__init__(model, model.coatnet.stages[-1].blocks[-1].mlp)

    def __call__(self, input_tensor):
        pred, class_probs = self.get_cam(input_tensor)

        # For CoAtNet, we need to reshape the activations to spatial dimensions
        b, n, c = self.activations.shape
        h = w = int(np.sqrt(n))

        activations = self.activations.reshape(b, h, w, c).permute(0, 3, 1, 2)
        gradients = self.gradients.reshape(b, h, w, c).permute(0, 3, 1, 2)

        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
        cam = torch.sum(weights * activations, dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = F.interpolate(cam, size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

        return cam.squeeze().cpu().numpy(), pred, class_probs

class XCiTGradCAM(BaseGradCAM):
    def __init__(self, model):
        super().__init__(model, model.xcit.blocks[-1].local_mp)

    def __call__(self, input_tensor):
        pred, class_probs = self.get_cam(input_tensor)

        # For XCiT, handle the local attention features
        if len(self.gradients.shape) == 3:
            b, n, c = self.gradients.shape
            h = w = int(np.sqrt(n))

            activations = self.activations.reshape(b, h, w, c).permute(0, 3, 1, 2)
            gradients = self.gradients.reshape(b, h, w, c).permute(0, 3, 1, 2)
        else:
            activations = self.activations
            gradients = self.gradients

        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)
        cam = torch.sum(weights * activations, dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = F.interpolate(cam, size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

        return cam.squeeze().cpu().numpy(), pred, class_probs

def visualize_all_models(image_path, models_dict, tumor_types, device='cuda'):
    """
    Visualize GradCAM for a single image across all models
    """
    # Prepare image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Initialize model-specific GradCAM
    gradcams = {}
    for model_name, model in models_dict.items():
        try:
            if model_name == 'efficientnet':
                gradcams[model_name] = EfficientNetGradCAM(model)
            elif model_name == 'levit':
                gradcams[model_name] = LeViTGradCAM(model)
            elif model_name == 'coatnet':
                gradcams[model_name] = CoAtNetGradCAM(model)
            elif model_name == 'xcit':
                gradcams[model_name] = XCiTGradCAM(model)
        except Exception as e:
            print(f"Error initializing {model_name}: {str(e)}")
            continue

    # Create visualization
    fig, axes = plt.subplots(1, len(gradcams) + 1, figsize=(5*(len(gradcams) + 1), 5))

    # Original image
    original_img = cv2.imread(image_path)
    original_img = cv2.resize(original_img, (224, 224))
    axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original')
    axes[0].axis('off')

    # Generate GradCAM for each model
    for idx, (model_name, gradcam) in enumerate(gradcams.items()):
        try:
            # Create a new input tensor for each model to avoid graph conflicts
            input_tensor = transform(image).unsqueeze(0).to(device)

            cam, pred_class, class_probs = gradcam(input_tensor)

            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
            superimposed = cv2.addWeighted(original_img, 0.6, heatmap, 0.4, 0)

            axes[idx + 1].imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
            axes[idx + 1].set_title(f'{model_name.upper()}\nPred: {tumor_types[pred_class]}\n' +
                                  '\n'.join([f'{tumor_types[i]}: {prob:.2%}'
                                           for i, prob in enumerate(class_probs)]))
            axes[idx + 1].axis('off')
        except Exception as e:
            print(f"Error processing {model_name}: {str(e)}")
            continue

    plt.tight_layout()
    plt.show()

In [ ]:
# Initialize models and gradcam visualizer
models_dict = {
    'efficientnet': cnn_model,
    'levit': levit_model,
    'coatnet': coatnet_model,
    'xcit': xcit_model
}

tumor_types = ['Normal', 'Glioma', 'Meningioma', 'Pituitary']

# Pick a test image
image_path = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing/glioma/Te-gl_0286.jpg'

# Visualize
visualize_all_models(image_path, models_dict, tumor_types)


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

def get_attention_maps(model, x):
    """
    Directly compute attention maps from LeViT layers
    """
    attention_maps = []

    # Process through patch embedding
    x = model.levit.patch_embed(x)

    # Process through each stage
    for stage in model.levit.stages:
        # Apply stage transformation
        x = stage(x)
        # Get attention pattern
        attention = x.mean(1)  # Average over channels
        attention_maps.append(attention)

    return attention_maps

def process_attention_maps(attention_maps, image_size=(224, 224)):
    """
    Process attention maps to match image size
    """
    processed_maps = []

    for attn_map in attention_maps:
        # Interpolate to match image size
        resized_map = F.interpolate(
            attn_map.unsqueeze(1),
            size=image_size,
            mode='bilinear',
            align_corners=False
        )

        # Normalize the map
        normalized_map = resized_map.squeeze()
        normalized_map = (normalized_map - normalized_map.min()) / (normalized_map.max() - normalized_map.min() + 1e-8)
        processed_maps.append(normalized_map.cpu().detach())

    return processed_maps

def visualize_attention(image_path, model, tumor_types):
    """
    Visualize attention patterns from LeViT transformer
    """
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])

    input_tensor = transform(image).unsqueeze(0)
    original_image = np.array(image.resize((224, 224)))

    # Move to same device as model
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)

    # Get model prediction and attention maps
    model.eval()
    with torch.no_grad():
        # Get attention maps
        attention_maps = get_attention_maps(model, input_tensor)

        # Get prediction
        output = model(input_tensor)
        probs = torch.nn.functional.softmax(output, dim=1)
        predicted_class = output.argmax(dim=1).item()

    # Process attention maps
    processed_maps = process_attention_maps(attention_maps)

    # Visualization
    num_maps = len(processed_maps)
    fig = plt.figure(figsize=(20, 10))

    # Plot original image
    plt.subplot(2, num_maps + 1, 1)
    plt.imshow(original_image)
    plt.title('Original Image')
    plt.axis('off')

    # Plot combined attention
    plt.subplot(2, num_maps + 1, num_maps + 2)
    combined_attention = torch.stack(processed_maps).mean(0)
    plt.imshow(original_image)
    plt.imshow(combined_attention.numpy(), cmap='jet', alpha=0.5)
    plt.title('Combined Attention')
    plt.axis('off')

    # Plot individual attention maps
    for i, attn_map in enumerate(processed_maps):
        # Heatmap
        plt.subplot(2, num_maps + 1, i + 2)
        plt.imshow(attn_map.numpy(), cmap='jet')
        plt.title(f'Stage {i+1} Attention')
        plt.axis('off')

        # Overlay
        plt.subplot(2, num_maps + 1, num_maps + i + 3)
        plt.imshow(original_image)
        plt.imshow(attn_map.numpy(), cmap='jet', alpha=0.5)
        plt.title(f'Stage {i+1} Overlay')
        plt.axis('off')

    if tumor_types:
        pred_label = tumor_types[predicted_class]
        pred_prob = probs[0][predicted_class].item()
        plt.suptitle(f'LeViT Attention Visualization\nPrediction: {pred_label} ({pred_prob:.3f})',
                    fontsize=16)

    plt.tight_layout()
    plt.show()

    return processed_maps, predicted_class, probs

# Print model structure to verify the layer names
print("Model structure:")
print(levit_model)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F

class FeatureExtractor:
    def __init__(self, model):
        self.model = model
        self.features = []

    def get_features(self, x):
        """Extract features through forward pass"""
        self.features = []

        # Process through stem
        x = self.model.levit.stem(x)
        self.features.append(x)

        # Process through stages
        for stage in self.model.levit.stages:
            x = stage(x)
            self.features.append(x)

        return self.features

def process_feature_map(feature, size=(224, 224)):
    """Process feature map for visualization"""
    # Move to CPU and detach
    feature = feature.cpu().detach()

    # Get mean activation across channels
    if len(feature.shape) == 4:  # [B, C, H, W]
        feature = feature.mean(1)
    elif len(feature.shape) == 3:  # [B, L, C]
        # Reshape to 2D
        L = feature.shape[1]
        H = W = int(np.sqrt(L))
        feature = feature.mean(-1).view(-1, H, W)

    # Take first item if batched
    if len(feature.shape) > 2:
        feature = feature[0]

    # Normalize
    feature = (feature - feature.min()) / (feature.max() - feature.min() + 1e-8)

    # Resize
    feature = F.interpolate(
        feature.unsqueeze(0).unsqueeze(0),
        size=size,
        mode='bilinear',
        align_corners=False
    ).squeeze()

    return feature.numpy()

def visualize_features(image_path, model, tumor_types=None):
    """Visualize model features"""
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])

    input_tensor = transform(image).unsqueeze(0)
    original_image = np.array(image.resize((224, 224)))

    # Move to device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)

    # Get features and prediction
    feature_extractor = FeatureExtractor(model)

    with torch.no_grad():
        features = feature_extractor.get_features(input_tensor)
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        predicted_class = output.argmax(dim=1).item()

    # Process features
    processed_features = [process_feature_map(f) for f in features]

    # Visualization
    num_features = len(processed_features)
    rows = (num_features + 3) // 4
    cols = min(4, num_features + 1)

    fig = plt.figure(figsize=(20, 5 * rows))

    # Original image
    plt.subplot(rows, cols, 1)
    plt.imshow(original_image)
    plt.title('Original Image')
    plt.axis('off')

    # Feature visualizations
    if processed_features:
        # Combined features
        combined_features = np.mean(processed_features, axis=0)
        plt.subplot(rows, cols, 2)
        plt.imshow(original_image)
        plt.imshow(combined_features, cmap='jet', alpha=0.5)
        plt.title('Combined Features')
        plt.axis('off')

        # Individual features
        for idx, feature in enumerate(processed_features):
            plt.subplot(rows, cols, idx + 3)
            plt.imshow(original_image)
            plt.imshow(feature, cmap='jet', alpha=0.5)
            plt.title(f'Stage {idx+1} Features')
            plt.axis('off')

    if tumor_types:
        pred_label = tumor_types[predicted_class]
        pred_prob = probs[0][predicted_class].item()
        plt.suptitle(f'LeViT Feature Visualization\nPrediction: {pred_label} ({pred_prob:.3f})',
                    fontsize=16)

    plt.tight_layout()
    plt.show()

    return processed_features, predicted_class, probs

def plot_individual_feature(image_path, model, feature_idx=None):
    """Plot a specific feature map in detail"""
    # Load image
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])

    input_tensor = transform(image).unsqueeze(0)
    original_image = np.array(image.resize((224, 224)))

    # Get features
    extractor = FeatureExtractor(model)
    with torch.no_grad():
        features = extractor.get_features(input_tensor)

    # Process feature
    if feature_idx is None:
        # Use combined features
        processed_features = [process_feature_map(f) for f in features]
        feature_map = np.mean(processed_features, axis=0)
        title = 'Combined Features'
    else:
        feature_map = process_feature_map(features[feature_idx])
        title = f'Stage {feature_idx + 1} Features'

    # Visualization
    plt.figure(figsize=(15, 5))

    plt.subplot(131)
    plt.imshow(original_image)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(feature_map, cmap='jet')
    plt.title(f'{title} Heatmap')
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(original_image)
    plt.imshow(feature_map, cmap='jet', alpha=0.5)
    plt.title(f'{title} Overlay')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Try running:
# features, pred_class, probabilities = visualize_features(
#     image_path='/content/drive/MyDrive/Colab Notebooks/Resized_Testing/glioma/Te-gl_0286.jpg',
#     model=levit_model,
#     tumor_types=tumor_types
# )

In [ ]:
features, pred_class, probabilities = visualize_features(
    image_path='/content/drive/MyDrive/Colab Notebooks/Resized_Testing/glioma/Te-gl_0286.jpg',
    model=levit_model,
    tumor_types=tumor_types
)

# To see a specific stage in detail:
plot_individual_feature(
    image_path='/content/drive/MyDrive/Colab Notebooks/Resized_Testing/glioma/Te-gl_0286.jpg',
    model=levit_model,
    feature_idx=1  # Try different indices
)

In [ ]:
# Load the image again for the heatmap
image = Image.open('/content/drive/MyDrive/Colab Notebooks/Resized_Testing/glioma/Te-gl_0286.jpg').convert('RGB')
original_image = np.array(image.resize((224, 224)))

# Plot combined attention heatmap
plot_attention_heatmap(attention_maps, original_image)

# Plot specific layer (e.g., layer 0)
plot_attention_heatmap(attention_maps, original_image, layer_idx=0)

# Saliency Maps

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import random

class SaliencyMap:
    def __init__(self, model):
        self.model = model
        self.model.eval()

    def generate_saliency_map(self, input_image, target_class):
        input_image.requires_grad_()

        # Forward pass
        output = self.model(input_image)

        # Zero all existing gradients
        self.model.zero_grad()

        # Calculate gradients of the target class score with respect to the input image
        target_score = output[0][target_class]
        target_score.backward()

        # Generate saliency map
        saliency_map = input_image.grad.data.abs().sum(dim=1).squeeze().cpu().numpy()

        # Normalize the saliency map
        saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())

        return saliency_map

def apply_saliency_map(image, saliency_map):
    saliency_map = cv2.resize(saliency_map, (image.shape[1], image.shape[0]))
    saliency_map = np.uint8(255 * saliency_map)
    heatmap = cv2.applyColorMap(saliency_map, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)
    return superimposed_img

def visualize_saliency(model, image_path, target_class, tumor_types):
    # Preprocess the image
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0)

    # Initialize SaliencyMap
    saliency = SaliencyMap(model)

    # Generate saliency map
    saliency_map = saliency.generate_saliency_map(input_tensor, target_class)

    # Apply saliency map to original image
    original_image = cv2.imread(image_path)
    if original_image is None:
        print(f"Error: Could not read image at {image_path}")
        return

    result = apply_saliency_map(original_image, saliency_map)

    # Display result
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title(f'Saliency Map: {tumor_types[target_class]}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def visualize_random_images(model, test_folder, tumor_types):
    for i, tumor_type in enumerate(tumor_types):
        tumor_folder = os.path.join(test_folder, tumor_type)
        image_files = [f for f in os.listdir(tumor_folder) if f.endswith('.jpg') or f.endswith('.png')]

        if image_files:
            random_image = random.choice(image_files)
            image_path = os.path.join(tumor_folder, random_image)
            visualize_saliency(model, image_path, i, tumor_types)
        else:
            print(f"No images found for {tumor_type}")

# Usage
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Assuming 'model' is your loaded ResNet50 model
visualize_random_images(model, test_folder, tumor_types)

In [ ]:
!pip install lime

In [ ]:
import torch
import numpy as np
import cv2
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import random
from lime import lime_image
from skimage.segmentation import mark_boundaries

class LIMEExplainer:
    def __init__(self, model, preprocess):
        self.model = model
        self.preprocess = preprocess

    def predict(self, images):
        batch = torch.stack([self.preprocess(Image.fromarray(img.astype('uint8'))) for img in images])
        logits = self.model(batch)
        probs = torch.nn.functional.softmax(logits, dim=1)
        return probs.detach().numpy()

    def explain(self, image, target_class):
        explainer = lime_image.LimeImageExplainer()
        explanation = explainer.explain_instance(
            image,
            self.predict,
            top_labels=5,
            hide_color=0,
            num_samples=1000
        )

        temp, mask = explanation.get_image_and_mask(
            target_class,
            positive_only=True,
            num_features=5,
            hide_rest=True
        )

        return mark_boundaries(temp / 255.0, mask)

def visualize_lime(model, image_path, target_class, tumor_types, preprocess):
    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0)

    # Initialize LIMEExplainer
    lime_explainer = LIMEExplainer(model, preprocess)

    # Generate LIME explanation
    image_array = np.array(img)
    lime_explanation = lime_explainer.explain(image_array, target_class)

    # Display results
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(lime_explanation)
    plt.title(f'LIME Explanation: {tumor_types[target_class]}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def visualize_random_images(model, test_folder, tumor_types, preprocess):
    for i, tumor_type in enumerate(tumor_types):
        tumor_folder = os.path.join(test_folder, tumor_type)
        image_files = [f for f in os.listdir(tumor_folder) if f.endswith('.jpg') or f.endswith('.png')]

        if image_files:
            random_image = random.choice(image_files)
            image_path = os.path.join(tumor_folder, random_image)
            visualize_lime(model, image_path, i, tumor_types, preprocess)
        else:
            print(f"No images found for {tumor_type}")

# Usage
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Define preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming 'model' is your loaded ResNet50 model
visualize_random_images(model, test_folder, tumor_types, preprocess)

In [ ]:
import torch
import numpy as np
import cv2
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import random

class IntegratedGradients:
    def __init__(self, model, preprocess):
        self.model = model
        self.preprocess = preprocess

    def generate_integrated_gradients(self, input_image, target_class, steps=50):
        self.model.eval()

        # Generate alpha values
        alphas = torch.linspace(0, 1, steps)

        # Generate scaled inputs
        scaled_inputs = [input_image * alpha for alpha in alphas]

        # Compute gradients
        grads = []
        for scaled_input in scaled_inputs:
            scaled_input.requires_grad_(True)
            output = self.model(scaled_input)
            score = output[0][target_class]
            self.model.zero_grad()
            score.backward()
            grads.append(scaled_input.grad.cpu().detach().numpy())
            scaled_input.requires_grad_(False)

        # Compute integral
        avg_grads = np.average(grads, axis=0)
        integrated_gradients = (input_image.cpu().detach().numpy() - scaled_inputs[0].cpu().detach().numpy()) * avg_grads

        return integrated_gradients[0].sum(axis=0)

def normalize_image(image):
    return (image - image.min()) / (image.max() - image.min())

def apply_colormap(image):
    color_image = cv2.applyColorMap(np.uint8(255 * image), cv2.COLORMAP_JET)
    return cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

def visualize_integrated_gradients(model, image_path, target_class, tumor_types, preprocess):
    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0)

    # Initialize IntegratedGradients
    ig = IntegratedGradients(model, preprocess)

    # Generate Integrated Gradients
    attributions = ig.generate_integrated_gradients(input_tensor, target_class)

    # Normalize and apply colormap
    normalized_attributions = normalize_image(attributions)
    attribution_heatmap = apply_colormap(normalized_attributions)

    # Resize heatmap to match original image size
    original_image = cv2.imread(image_path)
    attribution_heatmap = cv2.resize(attribution_heatmap, (original_image.shape[1], original_image.shape[0]))

    # Superimpose heatmap on original image
    superimposed_img = cv2.addWeighted(original_image, 0.6, attribution_heatmap, 0.4, 0)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(attribution_heatmap)
    plt.title('Integrated Gradients')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title(f'Superimposed: {tumor_types[target_class]}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def visualize_random_images(model, test_folder, tumor_types, preprocess):
    for i, tumor_type in enumerate(tumor_types):
        tumor_folder = os.path.join(test_folder, tumor_type)
        image_files = [f for f in os.listdir(tumor_folder) if f.endswith('.jpg') or f.endswith('.png')]

        if image_files:
            random_image = random.choice(image_files)
            image_path = os.path.join(tumor_folder, random_image)
            visualize_integrated_gradients(model, image_path, i, tumor_types, preprocess)
        else:
            print(f"No images found for {tumor_type}")

# Usage remains the same as before

# Usage
test_folder = '/content/drive/MyDrive/Colab Notebooks/Resized_Testing'
tumor_types = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Define preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming 'model' is your loaded ResNet50 model
visualize_random_images(model, test_folder, tumor_types, preprocess)

## Pandas Dataframe

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import f1_score
import os
import time

def train_and_evaluate_model(model, train_loader, val_loader, test_loader, num_epochs=1, patience=1):
    start_time = time.time()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

    best_val_loss = float('inf')
    best_val_acc = 0.0
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        # ... (training loop code here)

        model.eval()
        # ... (validation loop code here)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping!')
            break

    end_time = time.time()
    training_time = end_time - start_time

    # Evaluate on test set
    test_metrics = evaluate_model(model, test_loader)

    return model, best_val_acc, training_time, test_metrics

def evaluate_model(model, data_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    all_preds = []
    all_labels = []
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')
    avg_loss = total_loss / total

    return {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Cross-entropy Loss': avg_loss
    }

# Assuming you have your data loaders and model initialization set up
results = []
k_folds = 2

for fold in range(k_folds):
    model = initialize_model(num_classes)
    train_loader, val_loader = get_fold_loaders(fold)  # You'll need to implement this function
    model, val_acc, train_time, _ = train_and_evaluate_model(model, train_loader, val_loader, val_loader)
    results.append({
        'Fold': fold + 1,
        'Validation Accuracy': val_acc,
        'Training Time (s)': train_time
    })

# Final training on entire dataset
full_train_loader = get_full_train_loader()  # You'll need to implement this function
test_loader = get_test_loader()  # You'll need to implement this function
final_model = initialize_model(num_classes)
final_model, final_val_acc, final_train_time, final_test_metrics = train_and_evaluate_model(final_model, full_train_loader, test_loader, test_loader)

results.append({
    'Fold': 'Final',
    'Validation Accuracy': final_val_acc,
    'Training Time (s)': final_train_time,
    **final_test_metrics
})

# Create the DataFrame
df = pd.DataFrame(results)

# Add model size and number of parameters
model_path = '/content/drive/MyDrive/Colab Notebooks/final_mri_classification_model.pth'
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Size in MB
num_params = sum(p.numel() for p in final_model.parameters())

df.loc[df['Fold'] == 'Final', 'Number of Parameters'] = num_params
df.loc[df['Fold'] == 'Final', 'Model Size (MB)'] = model_size

# Reorder columns for better readability
column_order = ['Fold', 'Validation Accuracy', 'Accuracy', 'F1 Score', 'Cross-entropy Loss',
                'Training Time (s)', 'Number of Parameters', 'Model Size (MB)']
df = df.reindex(columns=[col for col in column_order if col in df.columns])

# Display the DataFrame
print(df.to_string(index=False))